In [1]:

import csv
import gzip
import os
import matplotlib as mpl
import scipy.io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import umap
from sklearn.cluster import Birch, AffinityPropagation, DBSCAN, MeanShift, SpectralClustering, AgglomerativeClustering, KMeans, estimate_bandwidth
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import kneighbors_graph
from itertools import cycle, islice
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
gmmAverage = 0
kmAverage = 0
birchAverage = 0
wardAverage = 0
affPropAverage = 0
aggAverage = 0
spectralAverage = 0
document = open('[PARTIAL CLASSIFICATION] PATIENT ID 100 SIMULATIONS.txt', 'w')
for b in range(1, 101):
    matrix_dir = "genesrna"
    mat = pd.read_csv("genesrna/dataset4.csv")
    mat = np.array(mat)
    datafile = 'dataset4.tpm.txt'
    df = pd.read_csv(datafile, sep='\t')
    df = df.set_index('gene_name')
    cells = list(df.columns.values)
    features_path = os.path.join(matrix_dir, "features4.txt")
    annotation  = pd.read_csv(features_path,sep='\t',header=None)
    annotation.columns = ['gene_names']
    barcodes_path = os.path.join(matrix_dir, "barcodes4.txt")
    barcodes = [line.strip() for line in open(barcodes_path, 'r')]
    mat = mat.T 

    low_expr_thr = 1000
    high_expr_thr = 1000000
    per_cell_sum = mat.sum(axis=1)
    per_gene_sum = mat.sum(axis=0)
    mat = mat[:, (per_gene_sum>=low_expr_thr) & (per_gene_sum<=high_expr_thr)]
    mean_exp = mat.mean(axis=0)
    std_exp = np.sqrt(mat.std(axis=0))
    CV = std_exp/mean_exp

    mat = mat[:, CV >= 0.6]

    cells_expression = mat.sum(axis=1)
    mat = mat[cells_expression>=100, :]
    mat = np.log(mat+1)
    U, S, V = np.linalg.svd(mat) 
    eigvals = S**2 / np.sum(S**2)  
    sing_vals = np.arange(len(mat)) + 1
    pca = PCA(n_components=100)
    pca.fit(mat)
    mat_reduce=pca.transform(mat)
    true_indices = {}
    true_coords = {}
    with open('mat_reduce.txt', 'w') as f:
        for i in mat_reduce:
            f.write("%s\n" % i)
    embedding = umap.UMAP(n_neighbors=30, min_dist=0.1, metric='euclidean').fit_transform(mat_reduce)

    for i in range (0, len(embedding)):
        true_indices.update({str(embedding[i]):cells[i]})
        true_coords.update({cells[i]:str(embedding[i]).strip()})
    with open('embedding.txt', 'w') as f:
        for e in embedding:
            f.write("%s\n" % e)

    params = {'quantile': .3, 'eps': .5, 'damping': .9, 'preference':-200, 'n_neighbors':30, 'n_clusters':13}
    bandwidth = estimate_bandwidth(embedding, quantile=params['quantile'])
    connectivity = kneighbors_graph(embedding, n_neighbors=params['n_neighbors'], include_self = False)
    ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
    ward = AgglomerativeClustering(n_clusters=params['n_clusters'], linkage='ward', connectivity=connectivity)
    spectral = SpectralClustering(n_clusters=params['n_clusters'], eigen_solver='arpack', affinity="nearest_neighbors")
    dbscan = DBSCAN(eps=params['eps'])
    affinity_propagation = AffinityPropagation(damping=params['damping'], preference=params['preference'])
    average_linkage = AgglomerativeClustering(linkage="average", affinity="cityblock", n_clusters=params['n_clusters'], connectivity=connectivity)
    birch = Birch(n_clusters=params['n_clusters'])
    gmm = GaussianMixture(n_components=params['n_clusters'], covariance_type='full')
    kmeans = KMeans(n_clusters = params['n_clusters'])
    clustering_algorithms = ( 
        ('AffinityPropagation', affinity_propagation),
        ('MeanShift', ms),
        ('SpectralClustering', spectral),
        ('Ward', ward),
        ('AgglomerativeClustering', average_linkage),
        ('DBSCAN', dbscan),
        ('Birch', birch),
        ('GaussianMixture', gmm),
        ('KMeans', kmeans))
    #now plot everything
    
    for idx, (name, algorithm) in enumerate(clustering_algorithms):
        algorithm.fit(embedding)
        if hasattr(algorithm, 'labels_'):
            y_pred = algorithm.labels_.astype(np.int)
        else:
            y_pred = algorithm.predict(embedding)
        colors = np.array(list(islice(cycle(['#377eb8', '#ff7f00', '#4daf4a',
                                                 '#f781bf', '#a65628', '#984ea3',
                                                 '#999999', '#e41a1c', '#dede00']),
                                          int(max(y_pred) + 1))))
            # add black color for outliers (if any)
        colors = np.append(colors, ["#000000"])

        if hasattr(algorithm, 'labels_'):
            labels = algorithm.labels_
        else:
            labels = algorithm.predict(embedding)

        #Then get the frequency count of the non-negative labels
        counts = np.bincount(labels[labels>=0])

       
    def FindTrueLabels(algorithm):
        algorithm_coords = {}
        if hasattr(algorithm, 'labels_'):
            y_pred = algorithm.labels_.astype(np.int)
        else:
            y_pred = algorithm.predict(embedding)
        for i in range(len(embedding)):
            #plt.scatter(embedding[i:i+1, 0], embedding[i:i+1, 1], s=4, color=color)
            algorithm_coords.update({str(embedding[i]):y_pred[i]})

        true_labels = {}
        for i in algorithm_coords:
            true_labels.update({true_indices[i]:algorithm_coords[i]})
        return true_labels
    def DataCollectionPatientTvP(algorithm, cluster_1, cluster_2, cluster_3, cluster_4, cluster_5, cluster_6, cluster_7, cluster_8, cluster_9, cluster_10, cluster_11, cluster_12, cluster_13, results):
        labels = FindTrueLabels(algorithm)

        barcodes1 = {}
        barcodes2 = {}
        barcodes3 = {}
        barcodes4 = {}
        barcodes5 = {}
        barcodes6 = {}
        barcodes7 = {}
        barcodes8 = {}
        barcodes9 = {}
        barcodes10 = {}
        barcodes11 = {}
        barcodes12 = {}
        barcodes13 = {}

        truthnum1 = {}



        for i in labels:
            if labels[i] == 0:
                barcodes1.update({i:labels[i]})
            elif labels[i] == 1:
                barcodes2.update({i:labels[i]})
            elif labels[i] == 2:
                barcodes3.update({i:labels[i]})
            elif labels[i] == 3:
                barcodes4.update({i:labels[i]})
            elif labels[i] == 4:
                barcodes5.update({i:labels[i]})
            elif labels[i] == 5:
                barcodes6.update({i:labels[i]})
            elif labels[i] == 6:
                barcodes7.update({i:labels[i]})
            elif labels[i] == 7:
                barcodes8.update({i:labels[i]})
            elif labels[i] == 8:
                barcodes9.update({i:labels[i]})
            elif labels[i] == 9:
                barcodes10.update({i:labels[i]})
            elif labels[i] == 10:
                barcodes11.update({i:labels[i]})
            elif labels[i] == 11:
                barcodes12.update({i:labels[i]})
            elif labels[i] == 12:
                barcodes13.update({i:labels[i]})

        with open(cluster_1, 'w') as filehandle:
            for listitem in barcodes1:
                filehandle.write('%s\n' % listitem)
        with open(cluster_2, 'w') as filehandle:
            for listitem in barcodes2:
                filehandle.write('%s\n' % listitem)
        with open(cluster_3, 'w') as filehandle:
            for listitem in barcodes3:
                filehandle.write('%s\n' % listitem)
        with open(cluster_4, 'w') as filehandle:
            for listitem in barcodes4:
                filehandle.write('%s\n' % listitem)
        with open(cluster_5, 'w') as filehandle:
            for listitem in barcodes5:
                filehandle.write('%s\n' % listitem)
        with open(cluster_6, 'w') as filehandle:
            for listitem in barcodes6:
                filehandle.write('%s\n' % listitem)
        with open(cluster_7, 'w') as filehandle:
            for listitem in barcodes7:
                filehandle.write('%s\n' % listitem)
        with open(cluster_8, 'w') as filehandle:
            for listitem in barcodes8:
                filehandle.write('%s\n' % listitem)
        with open(cluster_9, 'w') as filehandle:
            for listitem in barcodes9:
                filehandle.write('%s\n' % listitem)
        with open(cluster_10, 'w') as filehandle:
            for listitem in barcodes10:
                filehandle.write('%s\n' % listitem)
        with open(cluster_11, 'w') as filehandle:
            for listitem in barcodes11:
                filehandle.write('%s\n' % listitem)
        with open(cluster_12, 'w') as filehandle:
            for listitem in barcodes12:
                filehandle.write('%s\n' % listitem)
        with open(cluster_13, 'w') as filehandle:
            for listitem in barcodes13:
                filehandle.write('%s\n' % listitem)



        BC01 = open('BC01_PARTIALLY_MODIFIED.txt').readlines()
        BC01_line = []
        for lines in BC01:
            BC01_line.append(lines)

        BC02 = open('BC02_PARTIALLY_MODIFIED.txt').readlines()
        BC02_line = []
        for lines in BC02:
            BC02_line.append(lines)

        BC03 = open('BC03_PARTIALLY_MODIFIED.txt').readlines()
        BC03_line = []
        for lines in BC03:
            BC03_line.append(lines)

        BC04 = open('BC04_PARTIALLY_MODIFIED.txt').readlines()
        BC04_line = []
        for lines in BC04:
            BC04_line.append(lines)

        BC05 = open('BC05_PARTIALLY_MODIFIED.txt').readlines()
        BC05_line = []
        for lines in BC05:
            BC05_line.append(lines)

        BC06 = open('BC06_PARTIALLY_MODIFIED.txt').readlines()
        BC06_line = []
        for lines in BC06:
            BC06_line.append(lines)

        BC07 = open('BC07_PARTIALLY_MODIFIED.txt').readlines()
        BC07_line = []
        for lines in BC07:
            BC07_line.append(lines)

        BC08 = open('BC08_PARTIALLY_MODIFIED.txt').readlines()
        BC08_line = []
        for lines in BC08:
            BC08_line.append(lines)

        BC09 = open('BC09_PARTIALLY_MODIFIED.txt').readlines()
        BC09_line = []
        for lines in BC09:
            BC09_line.append(lines)

        BC10 = open('BC10_PARTIALLY_MODIFIED.txt').readlines()
        BC10_line = []
        for lines in BC10:
            BC10_line.append(lines)

        BC11 = open('BC11_PARTIALLY_MODIFIED.txt').readlines()
        BC11_line = []
        for lines in BC11:
            BC11_line.append(lines)

        BC07LN = open('BC07LN_PARTIALLY_MODIFIED.txt').readlines()
        BC07LN_line = []
        for lines in BC07LN:
            BC07LN_line.append(lines)

        BC03LN = open('BC03LN_PARTIALLY_MODIFIED.txt').readlines()
        BC03LN_line = []
        for lines in BC03LN:
            BC03LN_line.append(lines)

        res = open(results, "w")

        bc01count1 = 0
        bc02count1 = 0
        bc03count1 = 0
        bc03lncount1 = 0
        bc04count1 = 0
        bc05count1 = 0 
        bc06count1 = 0
        bc07count1 = 0
        bc07lncount1 = 0
        bc08count1 = 0 
        bc09count1 = 0
        bc10count1 = 0
        bc11count1 = 0

        bc01count2 = 0
        bc02count2 = 0
        bc03count2 = 0
        bc03lncount2 = 0
        bc04count2 = 0
        bc05count2 = 0 
        bc06count2 = 0
        bc07count2 = 0
        bc07lncount2 = 0
        bc08count2 = 0 
        bc09count2 = 0
        bc10count2 = 0
        bc11count2 = 0

        bc01count3 = 0
        bc02count3 = 0
        bc03count3 = 0
        bc03lncount3 = 0
        bc04count3 = 0
        bc05count3 = 0 
        bc06count3 = 0
        bc07count3 = 0
        bc07lncount3 = 0
        bc08count3 = 0 
        bc09count3 = 0
        bc10count3 = 0
        bc11count3 = 0

        bc01count4 = 0
        bc02count4 = 0
        bc03count4 = 0
        bc03lncount4 = 0
        bc04count4 = 0
        bc05count4 = 0 
        bc06count4 = 0
        bc07count4 = 0
        bc07lncount4 = 0
        bc08count4 = 0 
        bc09count4 = 0
        bc10count4 = 0
        bc11count4 = 0

        bc01count5 = 0
        bc02count5 = 0
        bc03count5 = 0
        bc03lncount5 = 0
        bc04count5 = 0
        bc05count5 = 0 
        bc06count5 = 0
        bc07count5 = 0
        bc07lncount5 = 0
        bc08count5 = 0 
        bc09count5 = 0
        bc10count5 = 0
        bc11count5 = 0

        bc01count6 = 0
        bc02count6 = 0
        bc03count6 = 0
        bc03lncount6 = 0
        bc04count6 = 0
        bc05count6 = 0 
        bc06count6 = 0
        bc07count6 = 0
        bc07lncount6 = 0
        bc08count6 = 0 
        bc09count6 = 0
        bc10count6 = 0
        bc11count6 = 0

        bc01count7 = 0
        bc02count7 = 0
        bc03count7 = 0
        bc03lncount7 = 0
        bc04count7 = 0
        bc05count7 = 0 
        bc06count7 = 0
        bc07count7 = 0
        bc07lncount7 = 0
        bc08count7 = 0 
        bc09count7 = 0
        bc10count7 = 0
        bc11count7 = 0

        bc01count8 = 0
        bc02count8 = 0
        bc03count8 = 0
        bc03lncount8 = 0
        bc04count8 = 0
        bc05count8 = 0 
        bc06count8 = 0
        bc07count8 = 0
        bc07lncount8 = 0
        bc08count8 = 0 
        bc09count8 = 0
        bc10count8 = 0
        bc11count8 = 0

        bc01count9 = 0
        bc02count9 = 0
        bc03count9 = 0
        bc03lncount9 = 0
        bc04count9 = 0
        bc05count9 = 0 
        bc06count9 = 0
        bc07count9 = 0
        bc07lncount9 = 0
        bc08count9 = 0 
        bc09count9 = 0
        bc10count9 = 0
        bc11count9 = 0

        bc01count10 = 0
        bc02count10 = 0
        bc03count10 = 0
        bc03lncount10 = 0
        bc04count10 = 0
        bc05count10 = 0 
        bc06count10 = 0
        bc07count10 = 0
        bc07lncount10 = 0
        bc08count10 = 0 
        bc09count10 = 0
        bc10count10 = 0
        bc11count10 = 0

        bc01count11 = 0
        bc02count11 = 0
        bc03count11 = 0
        bc03lncount11 = 0
        bc04count11 = 0
        bc05count11 = 0 
        bc06count11 = 0
        bc07count11 = 0
        bc07lncount11 = 0
        bc08count11 = 0 
        bc09count11 = 0
        bc10count11 = 0
        bc11count11 = 0

        bc01count12 = 0
        bc02count12 = 0
        bc03count12 = 0
        bc03lncount12 = 0
        bc04count12 = 0
        bc05count12 = 0 
        bc06count12 = 0
        bc07count12 = 0
        bc07lncount12 = 0
        bc08count12 = 0 
        bc09count12 = 0
        bc10count12 = 0
        bc11count12 = 0

        bc01count13 = 0
        bc02count13 = 0
        bc03count13 = 0
        bc03lncount13 = 0
        bc04count13 = 0
        bc05count13 = 0 
        bc06count13 = 0
        bc07count13 = 0
        bc07lncount13 = 0
        bc08count13 = 0 
        bc09count13 = 0
        bc10count13 = 0
        bc11count13 = 0


        ## CLUSTER 1
        cluster1 = open(cluster_1).readlines()
        cluster1_line = []
        for lines in cluster1:
            cluster1_line.append(lines)
        for i in cluster1_line:
            for j in BC01_line:
                if i == j:
                    bc01count1 = bc01count1 + 1
        res.write("BC01 Count Cluster 1: \n")
        res.write(str(bc01count1/(len(cluster1_line))) + " \n")
        res.write(str(bc01count1) + " \n")

        for x in cluster1_line:
            for y in BC02_line:
                if x == y:
                    bc02count1 = bc02count1 + 1
        res.write("BC02 Count Cluster 1: \n")
        res.write(str(bc02count1/(len(cluster1_line))) + "\n")
        res.write(str(bc02count1) + "\n")

        for x in cluster1_line:
            for y in BC03_line:
                if x == y:
                    bc03count1 = bc03count1 + 1
        res.write("BC03 Count Cluster 1: \n")
        res.write(str(bc03count1/(len(cluster1_line))) + "\n")
        res.write(str(bc03count1) + "\n")

        for x in cluster1_line:
            for y in BC03LN_line:
                if x == y:
                    bc03lncount1 = bc03lncount1 + 1
        res.write("BC03 Count Cluster 1: \n")
        res.write(str(bc03lncount1/(len(cluster1_line))) + "\n")
        res.write(str(bc03lncount1) + "\n")

        for x in cluster1_line:
            for y in BC04_line:
                if x == y:
                    bc04count1 = bc04count1 + 1
        res.write("BC04 Count Cluster 1: \n")
        res.write(str(bc04count1/(len(cluster1_line))) + "\n")
        res.write(str(bc04count1) + "\n")

        for x in cluster1_line:
            for y in BC05_line:
                if x == y:
                    bc05count1 = bc05count1 + 1
        res.write("BC05 Count Cluster 1: \n")
        res.write(str(bc05count1/(len(cluster1_line))) + "\n")
        res.write(str(bc05count1) + "\n")

        for x in cluster1_line:
            for y in BC06_line:
                if x == y:
                    bc06count1 = bc06count1 + 1
        res.write("BC06 Count Cluster 1: \n")
        res.write(str(bc06count1/(len(cluster1_line))) + "\n")
        res.write(str(bc06count1) + "\n")

        for x in cluster1_line:
            for y in BC07_line:
                if x == y:
                    bc07count1 = bc07count1 + 1
        res.write("BC07 Count Cluster 1: \n")
        res.write(str(bc07count1/(len(cluster1_line))) + "\n")
        res.write(str(bc07count1) + "\n")

        for x in cluster1_line:
            for y in BC07LN_line:
                if x == y:
                    bc07lncount1 = bc07lncount1 + 1
        res.write("BC07LN Count Cluster 1: \n")
        res.write(str(bc07lncount1/(len(cluster1_line))) + "\n")
        res.write(str(bc07lncount1) + "\n")

        for x in cluster1_line:
            for y in BC08_line:
                if x == y:
                    bc08count1 = bc08count1 + 1
        res.write("BC08 Count Cluster 1: \n")
        res.write(str(bc08count1/(len(cluster1_line))) + "\n")
        res.write(str(bc08count1) + "\n")

        for x in cluster1_line:
            for y in BC09_line:
                if x == y:
                    bc09count1 = bc09count1 + 1
        res.write("BC09 Count Cluster 1: \n")
        res.write(str(bc09count1/(len(cluster1_line))) + "\n")
        res.write(str(bc09count1) + "\n")

        for x in cluster1_line:
            for y in BC10_line:
                if x == y:
                    bc10count1 = bc10count1 + 1
        res.write("BC10 Count Cluster 1: \n")
        res.write(str(bc10count1/(len(cluster1_line))) + "\n")
        res.write(str(bc10count1) + "\n")

        for x in cluster1_line:
            for y in BC11_line:
                if x == y:
                    bc11count1 = bc11count1 + 1
        res.write("BC11 Count Cluster 1: \n")
        res.write(str(bc11count1/(len(cluster1_line))) + "\n")
        res.write(str(bc11count1) + "\n")


        ## CLUSTER 2

        cluster2 = open(cluster_2).readlines()
        cluster2_line = []
        for lines in cluster2:
            cluster2_line.append(lines)
        for i in cluster2_line:
            for j in BC01_line:
                if i == j:
                    bc01count2 = bc01count2 + 1
        res.write("BC01 Count Cluster 2: \n")
        res.write(str(bc01count2/(len(cluster2_line))) + " \n")
        res.write(str(bc01count2) + " \n")

        for x in cluster2_line:
            for y in BC02_line:
                if x == y:
                    bc02count2 = bc02count2 + 1
        res.write("BC02 Count Cluster 2: \n")
        res.write(str(bc02count2/(len(cluster2_line))) + "\n")
        res.write(str(bc02count2) + "\n")

        for x in cluster2_line:
            for y in BC03_line:
                if x == y:
                    bc03count2 = bc03count2 + 1
        res.write("BC03 Count Cluster 2: \n")
        res.write(str(bc03count2/(len(cluster2_line))) + "\n")
        res.write(str(bc03count2) + "\n")

        for x in cluster2_line:
            for y in BC03LN_line:
                if x == y:
                    bc03lncount2 = bc03lncount2 + 1
        res.write("BC03 Count Cluster 2: \n")
        res.write(str(bc03lncount2/(len(cluster2_line))) + "\n")
        res.write(str(bc03lncount2) + "\n")

        for x in cluster2_line:
            for y in BC04_line:
                if x == y:
                    bc04count2 = bc04count2 + 1
        res.write("BC04 Count Cluster 2: \n")
        res.write(str(bc04count2/(len(cluster2_line))) + "\n")
        res.write(str(bc04count2) + "\n")

        for x in cluster2_line:
            for y in BC05_line:
                if x == y:
                    bc05count2 = bc05count2 + 1
        res.write("BC05 Count Cluster 2: \n")
        res.write(str(bc05count2/(len(cluster2_line))) + "\n")
        res.write(str(bc05count2) + "\n")

        for x in cluster2_line:
            for y in BC06_line:
                if x == y:
                    bc06count2 = bc06count2 + 1
        res.write("BC06 Count Cluster 2: \n")
        res.write(str(bc06count2/(len(cluster2_line))) + "\n")
        res.write(str(bc06count2) + "\n")

        for x in cluster2_line:
            for y in BC07_line:
                if x == y:
                    bc07count2 = bc07count2 + 1
        res.write("BC07 Count Cluster 2: \n")
        res.write(str(bc07count2/(len(cluster2_line))) + "\n")
        res.write(str(bc07count2) + "\n")

        for x in cluster2_line:
            for y in BC07LN_line:
                if x == y:
                    bc07lncount2 = bc07lncount2 + 1
        res.write("BC07LN Count Cluster 2: \n")
        res.write(str(bc07lncount2/(len(cluster2_line))) + "\n")
        res.write(str(bc07lncount2) + "\n")

        for x in cluster2_line:
            for y in BC08_line:
                if x == y:
                    bc08count2 = bc08count2 + 1
        res.write("BC08 Count Cluster 2: \n")
        res.write(str(bc08count2/(len(cluster2_line))) + "\n")
        res.write(str(bc08count2) + "\n")

        for x in cluster2_line:
            for y in BC09_line:
                if x == y:
                    bc09count2 = bc09count2 + 1
        res.write("BC09 Count Cluster 2: \n")
        res.write(str(bc09count2/(len(cluster2_line))) + "\n")
        res.write(str(bc09count2) + "\n")

        for x in cluster2_line:
            for y in BC10_line:
                if x == y:
                    bc10count2 = bc10count2 + 1
        res.write("BC10 Count Cluster 2: \n")
        res.write(str(bc10count2/(len(cluster2_line))) + "\n")
        res.write(str(bc10count2) + "\n")

        for x in cluster2_line:
            for y in BC11_line:
                if x == y:
                    bc11count2 = bc11count2 + 1
        res.write("BC11 Count Cluster 2: \n")
        res.write(str(bc11count2/(len(cluster2_line))) + "\n")
        res.write(str(bc11count2) + "\n")


        ## CLUSTER 3

        cluster3 = open(cluster_3).readlines()
        cluster3_line = []
        for lines in cluster3:
            cluster3_line.append(lines)
        for i in cluster3_line:
            for j in BC01_line:
                if i == j:
                    bc01count3 = bc01count3 + 1
        res.write("BC01 Count Cluster 3: \n")
        res.write(str(bc01count3/(len(cluster3_line))) + " \n")
        res.write(str(bc01count3) + " \n")

        for x in cluster3_line:
            for y in BC02_line:
                if x == y:
                    bc02count3 = bc02count3 + 1
        res.write("BC02 Count Cluster 3: \n")
        res.write(str(bc02count3/(len(cluster3_line))) + "\n")
        res.write(str(bc02count3) + "\n")

        for x in cluster3_line:
            for y in BC03_line:
                if x == y:
                    bc03count3 = bc03count3 + 1
        res.write("BC03 Count Cluster 3: \n")
        res.write(str(bc03count3/(len(cluster3_line))) + "\n")
        res.write(str(bc03count3) + "\n")

        for x in cluster3_line:
            for y in BC03LN_line:
                if x == y:
                    bc03lncount3 = bc03lncount3 + 1
        res.write("BC03 Count Cluster 3: \n")
        res.write(str(bc03lncount3/(len(cluster3_line))) + "\n")
        res.write(str(bc03lncount3) + "\n")

        for x in cluster3_line:
            for y in BC04_line:
                if x == y:
                    bc04count3 = bc04count3 + 1
        res.write("BC04 Count Cluster 3: \n")
        res.write(str(bc04count3/(len(cluster3_line))) + "\n")
        res.write(str(bc04count3) + "\n")

        for x in cluster3_line:
            for y in BC05_line:
                if x == y:
                    bc05count3 = bc05count3 + 1
        res.write("BC05 Count Cluster 3: \n")
        res.write(str(bc05count3/(len(cluster3_line))) + "\n")
        res.write(str(bc05count3) + "\n")

        for x in cluster3_line:
            for y in BC06_line:
                if x == y:
                    bc06count3 = bc06count3 + 1
        res.write("BC06 Count Cluster 3: \n")
        res.write(str(bc06count3/(len(cluster3_line))) + "\n")
        res.write(str(bc06count3) + "\n")

        for x in cluster3_line:
            for y in BC07_line:
                if x == y:
                    bc07count3 = bc07count3 + 1
        res.write("BC07 Count Cluster 3: \n")
        res.write(str(bc07count3/(len(cluster3_line))) + "\n")
        res.write(str(bc07count3) + "\n")

        for x in cluster3_line:
            for y in BC07LN_line:
                if x == y:
                    bc07lncount3 = bc07lncount3 + 1
        res.write("BC07LN Count Cluster 3: \n")
        res.write(str(bc07lncount3/(len(cluster3_line))) + "\n")
        res.write(str(bc07lncount3) + "\n")

        for x in cluster3_line:
            for y in BC08_line:
                if x == y:
                    bc08count3 = bc08count3 + 1
        res.write("BC08 Count Cluster 3: \n")
        res.write(str(bc08count3/(len(cluster3_line))) + "\n")
        res.write(str(bc08count3) + "\n")

        for x in cluster3_line:
            for y in BC09_line:
                if x == y:
                    bc09count3 = bc09count3 + 1
        res.write("BC09 Count Cluster 3: \n")
        res.write(str(bc09count3/(len(cluster3_line))) + "\n")
        res.write(str(bc09count3) + "\n")

        for x in cluster3_line:
            for y in BC10_line:
                if x == y:
                    bc10count3 = bc10count3 + 1
        res.write("BC10 Count Cluster 3: \n")
        res.write(str(bc10count3/(len(cluster3_line))) + "\n")
        res.write(str(bc10count3) + "\n")

        for x in cluster3_line:
            for y in BC11_line:
                if x == y:
                    bc11count3 = bc11count3 + 1
        res.write("BC11 Count Cluster 3: \n")
        res.write(str(bc11count3/(len(cluster3_line))) + "\n")
        res.write(str(bc11count3) + "\n")



        ## CLUSTER 4

        cluster4 = open(cluster_4).readlines()
        cluster4_line = []
        for lines in cluster4:
            cluster4_line.append(lines)
        for i in cluster4_line:
            for j in BC01_line:
                if i == j:
                    bc01count4 = bc01count4 + 1
        res.write("BC01 Count Cluster 4: \n")
        res.write(str(bc01count4/(len(cluster4_line))) + " \n")
        res.write(str(bc01count4) + " \n")

        for x in cluster4_line:
            for y in BC02_line:
                if x == y:
                    bc02count4 = bc02count4 + 1
        res.write("BC02 Count Cluster 4: \n")
        res.write(str(bc02count4/(len(cluster4_line))) + "\n")
        res.write(str(bc02count4) + "\n")

        for x in cluster4_line:
            for y in BC03_line:
                if x == y:
                    bc03count4 = bc03count4 + 1
        res.write("BC03 Count Cluster 4: \n")
        res.write(str(bc03count4/(len(cluster4_line))) + "\n")
        res.write(str(bc03count4) + "\n")

        for x in cluster4_line:
            for y in BC03LN_line:
                if x == y:
                    bc03lncount4 = bc03lncount4 + 1
        res.write("BC03 Count Cluster 4: \n")
        res.write(str(bc03lncount4/(len(cluster4_line))) + "\n")
        res.write(str(bc03lncount4) + "\n")

        for x in cluster4_line:
            for y in BC04_line:
                if x == y:
                    bc04count4 = bc04count4 + 1
        res.write("BC04 Count Cluster 4: \n")
        res.write(str(bc04count4/(len(cluster4_line))) + "\n")
        res.write(str(bc04count4) + "\n")

        for x in cluster4_line:
            for y in BC05_line:
                if x == y:
                    bc05count4 = bc05count4 + 1
        res.write("BC05 Count Cluster 4: \n")
        res.write(str(bc05count4/(len(cluster4_line))) + "\n")
        res.write(str(bc05count4) + "\n")

        for x in cluster4_line:
            for y in BC06_line:
                if x == y:
                    bc06count4 = bc06count4 + 1
        res.write("BC06 Count Cluster 4: \n")
        res.write(str(bc06count4/(len(cluster4_line))) + "\n")
        res.write(str(bc06count4) + "\n")

        for x in cluster4_line:
            for y in BC07_line:
                if x == y:
                    bc07count4 = bc07count4 + 1
        res.write("BC07 Count Cluster 4: \n")
        res.write(str(bc07count4/(len(cluster4_line))) + "\n")
        res.write(str(bc07count4) + "\n")

        for x in cluster4_line:
            for y in BC07LN_line:
                if x == y:
                    bc07lncount4 = bc07lncount4 + 1
        res.write("BC07LN Count Cluster 4: \n")
        res.write(str(bc07lncount4/(len(cluster4_line))) + "\n")
        res.write(str(bc07lncount4) + "\n")

        for x in cluster4_line:
            for y in BC08_line:
                if x == y:
                    bc08count4 = bc08count4 + 1
        res.write("BC08 Count Cluster 4: \n")
        res.write(str(bc08count4/(len(cluster4_line))) + "\n")
        res.write(str(bc08count4) + "\n")

        for x in cluster4_line:
            for y in BC09_line:
                if x == y:
                    bc09count4 = bc09count4 + 1
        res.write("BC09 Count Cluster 4: \n")
        res.write(str(bc09count4/(len(cluster4_line))) + "\n")
        res.write(str(bc09count4) + "\n")

        for x in cluster4_line:
            for y in BC10_line:
                if x == y:
                    bc10count4 = bc10count4 + 1
        res.write("BC10 Count Cluster 4: \n")
        res.write(str(bc10count4/(len(cluster4_line))) + "\n")
        res.write(str(bc10count4) + "\n")

        for x in cluster4_line:
            for y in BC11_line:
                if x == y:
                    bc11count4 = bc11count4 + 1
        res.write("BC11 Count Cluster 4: \n")
        res.write(str(bc11count4/(len(cluster4_line))) + "\n")
        res.write(str(bc11count4) + "\n")



        ## CLUSTER 5

        cluster5 = open(cluster_5).readlines()
        cluster5_line = []
        for lines in cluster5:
            cluster5_line.append(lines)
        for i in cluster5_line:
            for j in BC01_line:
                if i == j:
                    bc01count5 = bc01count5 + 1
        res.write("BC01 Count Cluster 5: \n")
        res.write(str(bc01count5/(len(cluster5_line))) + " \n")
        res.write(str(bc01count5) + " \n")

        for x in cluster5_line:
            for y in BC02_line:
                if x == y:
                    bc02count5 = bc02count5 + 1
        res.write("BC02 Count Cluster 5: \n")
        res.write(str(bc02count5/(len(cluster5_line))) + "\n")
        res.write(str(bc02count5) + "\n")

        for x in cluster5_line:
            for y in BC03_line:
                if x == y:
                    bc03count5 = bc03count5 + 1
        res.write("BC03 Count Cluster 5: \n")
        res.write(str(bc03count5/(len(cluster5_line))) + "\n")
        res.write(str(bc03count5) + "\n")

        for x in cluster5_line:
            for y in BC03LN_line:
                if x == y:
                    bc03lncount5 = bc03lncount5 + 1
        res.write("BC03 Count Cluster 5: \n")
        res.write(str(bc03lncount5/(len(cluster5_line))) + "\n")
        res.write(str(bc03lncount5) + "\n")

        for x in cluster5_line:
            for y in BC04_line:
                if x == y:
                    bc04count5 = bc04count5 + 1
        res.write("BC04 Count Cluster 5: \n")
        res.write(str(bc04count5/(len(cluster5_line))) + "\n")
        res.write(str(bc04count5) + "\n")

        for x in cluster5_line:
            for y in BC05_line:
                if x == y:
                    bc05count5 = bc05count5 + 1
        res.write("BC05 Count Cluster 5: \n")
        res.write(str(bc05count5/(len(cluster5_line))) + "\n")
        res.write(str(bc05count5) + "\n")

        for x in cluster5_line:
            for y in BC06_line:
                if x == y:
                    bc06count5 = bc06count5 + 1
        res.write("BC06 Count Cluster 5: \n")
        res.write(str(bc06count5/(len(cluster5_line))) + "\n")
        res.write(str(bc06count5) + "\n")

        for x in cluster5_line:
            for y in BC07_line:
                if x == y:
                    bc07count5 = bc07count5 + 1
        res.write("BC07 Count Cluster 5: \n")
        res.write(str(bc07count5/(len(cluster5_line))) + "\n")
        res.write(str(bc07count5) + "\n")

        for x in cluster5_line:
            for y in BC07LN_line:
                if x == y:
                    bc07lncount5 = bc07lncount5 + 1
        res.write("BC07LN Count Cluster 5: \n")
        res.write(str(bc07lncount5/(len(cluster5_line))) + "\n")
        res.write(str(bc07lncount5) + "\n")

        for x in cluster5_line:
            for y in BC08_line:
                if x == y:
                    bc08count5 = bc08count5 + 1
        res.write("BC08 Count Cluster 5: \n")
        res.write(str(bc08count5/(len(cluster5_line))) + "\n")
        res.write(str(bc08count5) + "\n")

        for x in cluster5_line:
            for y in BC09_line:
                if x == y:
                    bc09count5 = bc09count5 + 1
        res.write("BC09 Count Cluster 5: \n")
        res.write(str(bc09count5/(len(cluster5_line))) + "\n")
        res.write(str(bc09count5) + "\n")

        for x in cluster5_line:
            for y in BC10_line:
                if x == y:
                    bc10count5 = bc10count5 + 1
        res.write("BC10 Count Cluster 5: \n")
        res.write(str(bc10count5/(len(cluster5_line))) + "\n")
        res.write(str(bc10count5) + "\n")

        for x in cluster5_line:
            for y in BC11_line:
                if x == y:
                    bc11count5 = bc11count5 + 1
        res.write("BC11 Count Cluster 5: \n")
        res.write(str(bc11count5/(len(cluster5_line))) + "\n")
        res.write(str(bc11count5) + "\n")



        ## CLUSTER 6

        cluster6 = open(cluster_6).readlines()
        cluster6_line = []
        for lines in cluster6:
            cluster6_line.append(lines)
        for i in cluster6_line:
            for j in BC01_line:
                if i == j:
                    bc01count6 = bc01count6 + 1
        res.write("BC01 Count Cluster 6: \n")
        res.write(str(bc01count6/(len(cluster6_line))) + " \n")
        res.write(str(bc01count6) + " \n")

        for x in cluster6_line:
            for y in BC02_line:
                if x == y:
                    bc02count6 = bc02count6 + 1
        res.write("BC02 Count Cluster 6: \n")
        res.write(str(bc02count6/(len(cluster6_line))) + "\n")
        res.write(str(bc02count6) + "\n")

        for x in cluster6_line:
            for y in BC03_line:
                if x == y:
                    bc03count6 = bc03count6 + 1
        res.write("BC03 Count Cluster 6: \n")
        res.write(str(bc03count6/(len(cluster6_line))) + "\n")
        res.write(str(bc03count6) + "\n")

        for x in cluster6_line:
            for y in BC03LN_line:
                if x == y:
                    bc03lncount6 = bc03lncount6 + 1
        res.write("BC03 Count Cluster 6: \n")
        res.write(str(bc03lncount6/(len(cluster6_line))) + "\n")
        res.write(str(bc03lncount6) + "\n")

        for x in cluster6_line:
            for y in BC04_line:
                if x == y:
                    bc04count6 = bc04count6 + 1
        res.write("BC04 Count Cluster 6: \n")
        res.write(str(bc04count6/(len(cluster6_line))) + "\n")
        res.write(str(bc04count6) + "\n")

        for x in cluster6_line:
            for y in BC05_line:
                if x == y:
                    bc05count6 = bc05count6 + 1
        res.write("BC05 Count Cluster 6: \n")
        res.write(str(bc05count6/(len(cluster6_line))) + "\n")
        res.write(str(bc05count6) + "\n")

        for x in cluster6_line:
            for y in BC06_line:
                if x == y:
                    bc06count6 = bc06count6 + 1
        res.write("BC06 Count Cluster 6: \n")
        res.write(str(bc06count6/(len(cluster6_line))) + "\n")
        res.write(str(bc06count6) + "\n")

        for x in cluster6_line:
            for y in BC07_line:
                if x == y:
                    bc07count6 = bc07count6 + 1
        res.write("BC07 Count Cluster 6: \n")
        res.write(str(bc07count6/(len(cluster6_line))) + "\n")
        res.write(str(bc07count6) + "\n")

        for x in cluster6_line:
            for y in BC07LN_line:
                if x == y:
                    bc07lncount6 = bc07lncount6 + 1
        res.write("BC07LN Count Cluster 6: \n")
        res.write(str(bc07lncount6/(len(cluster6_line))) + "\n")
        res.write(str(bc07lncount6) + "\n")

        for x in cluster6_line:
            for y in BC08_line:
                if x == y:
                    bc08count6 = bc08count6 + 1
        res.write("BC08 Count Cluster 6: \n")
        res.write(str(bc08count6/(len(cluster6_line))) + "\n")
        res.write(str(bc08count6) + "\n")

        for x in cluster6_line:
            for y in BC09_line:
                if x == y:
                    bc09count6 = bc09count6 + 1
        res.write("BC09 Count Cluster 6: \n")
        res.write(str(bc09count6/(len(cluster6_line))) + "\n")
        res.write(str(bc09count6) + "\n")

        for x in cluster6_line:
            for y in BC10_line:
                if x == y:
                    bc10count6 = bc10count6 + 1
        res.write("BC10 Count Cluster 6: \n")
        res.write(str(bc10count6/(len(cluster6_line))) + "\n")
        res.write(str(bc10count6) + "\n")

        for x in cluster6_line:
            for y in BC11_line:
                if x == y:
                    bc11count6 = bc11count6 + 1
        res.write("BC11 Count Cluster 6: \n")
        res.write(str(bc11count6/(len(cluster6_line))) + "\n")
        res.write(str(bc11count6) + "\n")


        ## CLUSTER 7

        cluster7 = open(cluster_7).readlines()
        cluster7_line = []
        for lines in cluster7:
            cluster7_line.append(lines)
        for i in cluster7_line:
            for j in BC01_line:
                if i == j:
                    bc01count7 = bc01count7 + 1
        res.write("BC01 Count Cluster 7: \n")
        res.write(str(bc01count7/(len(cluster7_line))) + " \n")
        res.write(str(bc01count7) + " \n")

        for x in cluster7_line:
            for y in BC02_line:
                if x == y:
                    bc02count7 = bc02count7 + 1
        res.write("BC02 Count Cluster 7: \n")
        res.write(str(bc02count7/(len(cluster7_line))) + "\n")
        res.write(str(bc02count7) + "\n")

        for x in cluster7_line:
            for y in BC03_line:
                if x == y:
                    bc03count7 = bc03count7 + 1
        res.write("BC03 Count Cluster 7: \n")
        res.write(str(bc03count7/(len(cluster7_line))) + "\n")
        res.write(str(bc03count7) + "\n")

        for x in cluster7_line:
            for y in BC03LN_line:
                if x == y:
                    bc03lncount7 = bc03lncount7 + 1
        res.write("BC03 Count Cluster 7: \n")
        res.write(str(bc03lncount7/(len(cluster7_line))) + "\n")
        res.write(str(bc03lncount7) + "\n")

        for x in cluster7_line:
            for y in BC04_line:
                if x == y:
                    bc04count7 = bc04count7 + 1
        res.write("BC04 Count Cluster 7: \n")
        res.write(str(bc04count7/(len(cluster7_line))) + "\n")
        res.write(str(bc04count7) + "\n")

        for x in cluster7_line:
            for y in BC06_line:
                if x == y:
                    bc06count7 = bc06count7 + 1
        res.write("BC06 Count Cluster 7: \n")
        res.write(str(bc06count7/(len(cluster7_line))) + "\n")
        res.write(str(bc06count7) + "\n")

        for x in cluster7_line:
            for y in BC07_line:
                if x == y:
                    bc07count7 = bc07count7 + 1
        res.write("BC07 Count Cluster 7: \n")
        res.write(str(bc07count7/(len(cluster7_line))) + "\n")
        res.write(str(bc07count7) + "\n")

        for x in cluster7_line:
            for y in BC07_line:
                if x == y:
                    bc07count7 = bc07count7 + 1
        res.write("BC07 Count Cluster 7: \n")
        res.write(str(bc07count7/(len(cluster7_line))) + "\n")
        res.write(str(bc07count7) + "\n")

        for x in cluster7_line:
            for y in BC07LN_line:
                if x == y:
                    bc07lncount7 = bc07lncount7 + 1
        res.write("BC07LN Count Cluster 7: \n")
        res.write(str(bc07lncount7/(len(cluster7_line))) + "\n")
        res.write(str(bc07lncount7) + "\n")

        for x in cluster7_line:
            for y in BC08_line:
                if x == y:
                    bc08count7 = bc08count7 + 1
        res.write("BC08 Count Cluster 7: \n")
        res.write(str(bc08count7/(len(cluster7_line))) + "\n")
        res.write(str(bc08count7) + "\n")

        for x in cluster7_line:
            for y in BC09_line:
                if x == y:
                    bc09count7 = bc09count7 + 1
        res.write("BC09 Count Cluster 7: \n")
        res.write(str(bc09count7/(len(cluster7_line))) + "\n")
        res.write(str(bc09count7) + "\n")

        for x in cluster7_line:
            for y in BC10_line:
                if x == y:
                    bc10count7 = bc10count7 + 1
        res.write("BC10 Count Cluster 7: \n")
        res.write(str(bc10count7/(len(cluster7_line))) + "\n")
        res.write(str(bc10count7) + "\n")

        for x in cluster7_line:
            for y in BC11_line:
                if x == y:
                    bc11count7 = bc11count7 + 1
        res.write("BC11 Count Cluster 7: \n")
        res.write(str(bc11count7/(len(cluster7_line))) + "\n")
        res.write(str(bc11count7) + "\n")


        ## CLUSTER 8

        cluster8 = open(cluster_8).readlines()
        cluster8_line = []
        for lines in cluster8:
            cluster8_line.append(lines)
        for i in cluster8_line:
            for j in BC01_line:
                if i == j:
                    bc01count8 = bc01count8 + 1
        res.write("BC01 Count Cluster 8: \n")
        res.write(str(bc01count8/(len(cluster8_line))) + " \n")
        res.write(str(bc01count8) + " \n")

        for x in cluster8_line:
            for y in BC02_line:
                if x == y:
                    bc02count8 = bc02count8 + 1
        res.write("BC02 Count Cluster 8: \n")
        res.write(str(bc02count8/(len(cluster8_line))) + "\n")
        res.write(str(bc02count8) + "\n")

        for x in cluster8_line:
            for y in BC03_line:
                if x == y:
                    bc03count8 = bc03count8 + 1
        res.write("BC03 Count Cluster 8: \n")
        res.write(str(bc03count8/(len(cluster8_line))) + "\n")
        res.write(str(bc03count8) + "\n")

        for x in cluster8_line:
            for y in BC03LN_line:
                if x == y:
                    bc03lncount8 = bc03lncount8 + 1
        res.write("BC03 Count Cluster 8: \n")
        res.write(str(bc03lncount8/(len(cluster8_line))) + "\n")
        res.write(str(bc03lncount8) + "\n")

        for x in cluster8_line:
            for y in BC04_line:
                if x == y:
                    bc04count8 = bc04count8 + 1
        res.write("BC04 Count Cluster 8: \n")
        res.write(str(bc04count8/(len(cluster8_line))) + "\n")
        res.write(str(bc04count8) + "\n")

        for x in cluster8_line:
            for y in BC05_line:
                if x == y:
                    bc05count8 = bc05count8 + 1
        res.write("BC05 Count Cluster 8: \n")
        res.write(str(bc05count8/(len(cluster8_line))) + "\n")
        res.write(str(bc05count8) + "\n")

        for x in cluster8_line:
            for y in BC06_line:
                if x == y:
                    bc06count8 = bc06count8 + 1
        res.write("BC06 Count Cluster 8: \n")
        res.write(str(bc06count8/(len(cluster8_line))) + "\n")
        res.write(str(bc06count8) + "\n")

        for x in cluster8_line:
            for y in BC07_line:
                if x == y:
                    bc07count8 = bc07count8 + 1
        res.write("BC07 Count Cluster 8: \n")
        res.write(str(bc07count8/(len(cluster8_line))) + "\n")
        res.write(str(bc07count8) + "\n")

        for x in cluster8_line:
            for y in BC07LN_line:
                if x == y:
                    bc07lncount8 = bc07lncount8 + 1
        res.write("BC07LN Count Cluster 8: \n")
        res.write(str(bc07lncount8/(len(cluster8_line))) + "\n")
        res.write(str(bc07lncount8) + "\n")

        for x in cluster8_line:
            for y in BC08_line:
                if x == y:
                    bc08count8 = bc08count8 + 1
        res.write("BC08 Count Cluster 8: \n")
        res.write(str(bc08count8/(len(cluster8_line))) + "\n")
        res.write(str(bc08count8) + "\n")

        for x in cluster8_line:
            for y in BC09_line:
                if x == y:
                    bc09count8 = bc09count8 + 1
        res.write("BC09 Count Cluster 8: \n")
        res.write(str(bc09count8/(len(cluster8_line))) + "\n")
        res.write(str(bc09count8) + "\n")

        for x in cluster8_line:
            for y in BC10_line:
                if x == y:
                    bc10count8 = bc10count8 + 1
        res.write("BC10 Count Cluster 8: \n")
        res.write(str(bc10count8/(len(cluster8_line))) + "\n")
        res.write(str(bc10count8) + "\n")

        for x in cluster8_line:
            for y in BC11_line:
                if x == y:
                    bc11count8 = bc11count8 + 1
        res.write("BC11 Count Cluster 8: \n")
        res.write(str(bc11count8/(len(cluster8_line))) + "\n")
        res.write(str(bc11count8) + "\n")




        ## CLUSTER 9

        cluster9 = open(cluster_9).readlines()
        cluster9_line = []
        for lines in cluster9:
            cluster9_line.append(lines)
        for i in cluster9_line:
            for j in BC01_line:
                if i == j:
                    bc01count9 = bc01count9 + 1
        res.write("BC01 Count Cluster 9: \n")
        res.write(str(bc01count9/(len(cluster9_line))) + " \n")
        res.write(str(bc01count9) + " \n")

        for x in cluster9_line:
            for y in BC02_line:
                if x == y:
                    bc02count9 = bc02count9 + 1
        res.write("BC02 Count Cluster 9: \n")
        res.write(str(bc02count9/(len(cluster9_line))) + "\n")
        res.write(str(bc02count9) + "\n")

        for x in cluster9_line:
            for y in BC03_line:
                if x == y:
                    bc03count9 = bc03count9 + 1
        res.write("BC03 Count Cluster 9: \n")
        res.write(str(bc03count9/(len(cluster9_line))) + "\n")
        res.write(str(bc03count9) + "\n")

        for x in cluster9_line:
            for y in BC03LN_line:
                if x == y:
                    bc03lncount9 = bc03lncount9 + 1
        res.write("BC03 Count Cluster 9: \n")
        res.write(str(bc03lncount9/(len(cluster9_line))) + "\n")
        res.write(str(bc03lncount9) + "\n")

        for x in cluster9_line:
            for y in BC04_line:
                if x == y:
                    bc04count9 = bc04count9 + 1
        res.write("BC04 Count Cluster 9: \n")
        res.write(str(bc04count9/(len(cluster9_line))) + "\n")
        res.write(str(bc04count9) + "\n")

        for x in cluster9_line:
            for y in BC05_line:
                if x == y:
                    bc05count9 = bc05count9 + 1
        res.write("BC05 Count Cluster 9: \n")
        res.write(str(bc05count9/(len(cluster9_line))) + "\n")
        res.write(str(bc05count9) + "\n")

        for x in cluster9_line:
            for y in BC06_line:
                if x == y:
                    bc06count9 = bc06count9 + 1
        res.write("BC06 Count Cluster 9: \n")
        res.write(str(bc06count9/(len(cluster9_line))) + "\n")
        res.write(str(bc06count9) + "\n")

        for x in cluster9_line:
            for y in BC07_line:
                if x == y:
                    bc07count9 = bc07count9 + 1
        res.write("BC07 Count Cluster 9: \n")
        res.write(str(bc07count9/(len(cluster9_line))) + "\n")
        res.write(str(bc07count9) + "\n")

        for x in cluster9_line:
            for y in BC07LN_line:
                if x == y:
                    bc07lncount9 = bc07lncount9 + 1
        res.write("BC07LN Count Cluster 9: \n")
        res.write(str(bc07lncount9/(len(cluster9_line))) + "\n")
        res.write(str(bc07lncount9) + "\n")

        for x in cluster9_line:
            for y in BC08_line:
                if x == y:
                    bc08count9 = bc08count9 + 1
        res.write("BC08 Count Cluster 9: \n")
        res.write(str(bc08count9/(len(cluster9_line))) + "\n")
        res.write(str(bc08count9) + "\n")

        for x in cluster9_line:
            for y in BC09_line:
                if x == y:
                    bc09count9 = bc09count9 + 1
        res.write("BC09 Count Cluster 9: \n")
        res.write(str(bc09count9/(len(cluster9_line))) + "\n")
        res.write(str(bc09count9) + "\n")

        for x in cluster9_line:
            for y in BC10_line:
                if x == y:
                    bc10count9 = bc10count9 + 1
        res.write("BC10 Count Cluster 9: \n")
        res.write(str(bc10count9/(len(cluster9_line))) + "\n")
        res.write(str(bc10count9) + "\n")

        for x in cluster9_line:
            for y in BC11_line:
                if x == y:
                    bc11count9 = bc11count9 + 1
        res.write("BC11 Count Cluster 9: \n")
        res.write(str(bc11count9/(len(cluster9_line))) + "\n")
        res.write(str(bc11count9) + "\n")



        ## CLUSTER 10

        cluster10 = open(cluster_10).readlines()
        cluster10_line = []
        for lines in cluster10:
            cluster10_line.append(lines)
        for i in cluster10_line:
            for j in BC01_line:
                if i == j:
                    bc01count10 = bc01count10 + 1
        res.write("BC01 Count Cluster 10: \n")
        res.write(str(bc01count10/(len(cluster10_line))) + " \n")
        res.write(str(bc01count10) + " \n")

        for x in cluster10_line:
            for y in BC02_line:
                if x == y:
                    bc02count10 = bc02count10 + 1
        res.write("BC02 Count Cluster 10: \n")
        res.write(str(bc02count10/(len(cluster10_line))) + "\n")
        res.write(str(bc02count10) + "\n")

        for x in cluster10_line:
            for y in BC03_line:
                if x == y:
                    bc03count10 = bc03count10 + 1
        res.write("BC03 Count Cluster 10: \n")
        res.write(str(bc03count10/(len(cluster10_line))) + "\n")
        res.write(str(bc03count10) + "\n")

        for x in cluster10_line:
            for y in BC03LN_line:
                if x == y:
                    bc03lncount10 = bc03lncount10 + 1
        res.write("BC03 Count Cluster 10: \n")
        res.write(str(bc03lncount10/(len(cluster10_line))) + "\n")
        res.write(str(bc03lncount10) + "\n")

        for x in cluster10_line:
            for y in BC04_line:
                if x == y:
                    bc04count10 = bc04count10 + 1
        res.write("BC04 Count Cluster 10: \n")
        res.write(str(bc04count10/(len(cluster10_line))) + "\n")
        res.write(str(bc04count10) + "\n")

        for x in cluster10_line:
            for y in BC05_line:
                if x == y:
                    bc05count10 = bc05count10 + 1
        res.write("BC05 Count Cluster 10: \n")
        res.write(str(bc05count10/(len(cluster10_line))) + "\n")
        res.write(str(bc05count10) + "\n")

        for x in cluster10_line:
            for y in BC06_line:
                if x == y:
                    bc06count10 = bc06count10 + 1
        res.write("BC06 Count Cluster 10: \n")
        res.write(str(bc06count10/(len(cluster10_line))) + "\n")
        res.write(str(bc06count10) + "\n")



        for x in cluster10_line:
            for y in BC07_line:
                if x == y:
                    bc07count10 = bc07count10 + 1
        res.write("BC07 Count Cluster 10: \n")
        res.write(str(bc07count10/(len(cluster10_line))) + "\n")
        res.write(str(bc07count10) + "\n")

        for x in cluster10_line:
            for y in BC07LN_line:
                if x == y:
                    bc07lncount10 = bc07lncount10 + 1
        res.write("BC07LN Count Cluster 10: \n")
        res.write(str(bc07lncount10/(len(cluster10_line))) + "\n")
        res.write(str(bc07lncount10) + "\n")

        for x in cluster10_line:
            for y in BC08_line:
                if x == y:
                    bc08count10 = bc08count10 + 1
        res.write("BC08 Count Cluster 10: \n")
        res.write(str(bc08count10/(len(cluster10_line))) + "\n")
        res.write(str(bc08count10) + "\n")

        for x in cluster10_line:
            for y in BC09_line:
                if x == y:
                    bc09count10 = bc09count10 + 1
        res.write("BC09 Count Cluster 10: \n")
        res.write(str(bc09count10/(len(cluster10_line))) + "\n")
        res.write(str(bc09count10) + "\n")

        for x in cluster10_line:
            for y in BC10_line:
                if x == y:
                    bc10count10 = bc10count10 + 1
        res.write("BC10 Count Cluster 10: \n")
        res.write(str(bc10count10/(len(cluster10_line))) + "\n")
        res.write(str(bc10count10) + "\n")

        for x in cluster10_line:
            for y in BC11_line:
                if x == y:
                    bc11count10 = bc11count10 + 1
        res.write("BC11 Count Cluster 10: \n")
        res.write(str(bc11count10/(len(cluster10_line))) + "\n")
        res.write(str(bc11count10) + "\n")




        ## CLUSTER 11

        cluster11 = open(cluster_11).readlines()
        cluster11_line = []
        for lines in cluster11:
            cluster11_line.append(lines)
        for i in cluster11_line:
            for j in BC01_line:
                if i == j:
                    bc01count11 = bc01count11 + 1
        res.write("BC01 Count Cluster 11: \n")
        res.write(str(bc01count11/(len(cluster11_line))) + " \n")
        res.write(str(bc01count11) + " \n")

        for x in cluster11_line:
            for y in BC02_line:
                if x == y:
                    bc02count11 = bc02count11 + 1
        res.write("BC02 Count Cluster 11: \n")
        res.write(str(bc02count11/(len(cluster11_line))) + "\n")
        res.write(str(bc02count11) + "\n")

        for x in cluster11_line:
            for y in BC03_line:
                if x == y:
                    bc03count11 = bc03count11 + 1
        res.write("BC03 Count Cluster 11: \n")
        res.write(str(bc03count11/(len(cluster11_line))) + "\n")
        res.write(str(bc03count11) + "\n")

        for x in cluster11_line:
            for y in BC03LN_line:
                if x == y:
                    bc03lncount11 = bc03lncount11 + 1
        res.write("BC03 Count Cluster 11: \n")
        res.write(str(bc03lncount11/(len(cluster11_line))) + "\n")
        res.write(str(bc03lncount11) + "\n")

        for x in cluster11_line:
            for y in BC04_line:
                if x == y:
                    bc04count11 = bc04count11 + 1
        res.write("BC04 Count Cluster 11: \n")
        res.write(str(bc04count11/(len(cluster11_line))) + "\n")
        res.write(str(bc04count11) + "\n")

        for x in cluster11_line:
            for y in BC05_line:
                if x == y:
                    bc05count11 = bc05count11 + 1
        res.write("BC05 Count Cluster 11: \n")
        res.write(str(bc05count11/(len(cluster11_line))) + "\n")
        res.write(str(bc05count11) + "\n")

        for x in cluster11_line:
            for y in BC06_line:
                if x == y:
                    bc06count11 = bc06count11 + 1
        res.write("BC06 Count Cluster 11: \n")
        res.write(str(bc06count11/(len(cluster11_line))) + "\n")
        res.write(str(bc06count11) + "\n")

        for x in cluster11_line:
            for y in BC07_line:
                if x == y:
                    bc07count11 = bc07count11 + 1
        res.write("BC07 Count Cluster 11: \n")
        res.write(str(bc07count11/(len(cluster11_line))) + "\n")
        res.write(str(bc07count11) + "\n")

        for x in cluster11_line:
            for y in BC07LN_line:
                if x == y:
                    bc07lncount11 = bc07lncount11 + 1
        res.write("BC07LN Count Cluster 11: \n")
        res.write(str(bc07lncount11/(len(cluster11_line))) + "\n")
        res.write(str(bc07lncount11) + "\n")

        for x in cluster11_line:
            for y in BC08_line:
                if x == y:
                    bc08count11 = bc08count11 + 1
        res.write("BC08 Count Cluster 11: \n")
        res.write(str(bc08count11/(len(cluster11_line))) + "\n")
        res.write(str(bc08count11) + "\n")

        for x in cluster11_line:
            for y in BC09_line:
                if x == y:
                    bc09count11 = bc09count11 + 1
        res.write("BC09 Count Cluster 11: \n")
        res.write(str(bc09count11/(len(cluster11_line))) + "\n")
        res.write(str(bc09count11) + "\n")

        for x in cluster11_line:
            for y in BC10_line:
                if x == y:
                    bc10count11 = bc10count11 + 1
        res.write("BC10 Count Cluster 11: \n")
        res.write(str(bc10count11/(len(cluster11_line))) + "\n")
        res.write(str(bc10count11) + "\n")

        for x in cluster11_line:
            for y in BC11_line:
                if x == y:
                    bc11count11 = bc11count11 + 1
        res.write("BC11 Count Cluster 11: \n")
        res.write(str(bc11count11/(len(cluster11_line))) + "\n")
        res.write(str(bc11count11) + "\n")






        ## CLUSTER 12

        cluster12 = open(cluster_12).readlines()
        cluster12_line = []
        for lines in cluster12:
            cluster12_line.append(lines)
        for i in cluster12_line:
            for j in BC01_line:
                if i == j:
                    bc01count12 = bc01count12 + 1
        res.write("BC01 Count Cluster 12: \n")
        res.write(str(bc01count12/(len(cluster12_line))) + " \n")
        res.write(str(bc01count12) + " \n")

        for x in cluster12_line:
            for y in BC02_line:
                if x == y:
                    bc02count12 = bc02count12 + 1
        res.write("BC02 Count Cluster 12: \n")
        res.write(str(bc02count12/(len(cluster12_line))) + "\n")
        res.write(str(bc02count12) + "\n")

        for x in cluster12_line:
            for y in BC03_line:
                if x == y:
                    bc03count12 = bc03count12 + 1
        res.write("BC03 Count Cluster 12: \n")
        res.write(str(bc03count12/(len(cluster12_line))) + "\n")
        res.write(str(bc03count12) + "\n")

        for x in cluster12_line:
            for y in BC03LN_line:
                if x == y:
                    bc03lncount12 = bc03lncount12 + 1
        res.write("BC03 Count Cluster 12: \n")
        res.write(str(bc03lncount12/(len(cluster12_line))) + "\n")
        res.write(str(bc03lncount12) + "\n")

        for x in cluster12_line:
            for y in BC04_line:
                if x == y:
                    bc04count12 = bc04count12 + 1
        res.write("BC04 Count Cluster 11: \n")
        res.write(str(bc04count12/(len(cluster12_line))) + "\n")
        res.write(str(bc04count12) + "\n")

        for x in cluster12_line:
            for y in BC05_line:
                if x == y:
                    bc05count12 = bc05count12 + 1
        res.write("BC05 Count Cluster 12: \n")
        res.write(str(bc05count12/(len(cluster12_line))) + "\n")
        res.write(str(bc05count12) + "\n")

        for x in cluster12_line:
            for y in BC06_line:
                if x == y:
                    bc06count12 = bc06count12 + 1
        res.write("BC06 Count Cluster 12: \n")
        res.write(str(bc06count12/(len(cluster12_line))) + "\n")
        res.write(str(bc06count12) + "\n")

        for x in cluster12_line:
            for y in BC07_line:
                if x == y:
                    bc07count12 = bc07count12 + 1
        res.write("BC07 Count Cluster 12: \n")
        res.write(str(bc07count12/(len(cluster12_line))) + "\n")
        res.write(str(bc07count12) + "\n")

        for x in cluster12_line:
            for y in BC07LN_line:
                if x == y:
                    bc07lncount12 = bc07lncount12 + 1
        res.write("BC07LN Count Cluster 12: \n")
        res.write(str(bc07lncount12/(len(cluster12_line))) + "\n")
        res.write(str(bc07lncount12) + "\n")

        for x in cluster12_line:
            for y in BC08_line:
                if x == y:
                    bc08count12 = bc08count12 + 1
        res.write("BC08 Count Cluster 12: \n")
        res.write(str(bc08count12/(len(cluster12_line))) + "\n")
        res.write(str(bc08count12) + "\n")

        for x in cluster12_line:
            for y in BC09_line:
                if x == y:
                    bc09count12 = bc09count12 + 1
        res.write("BC09 Count Cluster 12: \n")
        res.write(str(bc09count12/(len(cluster12_line))) + "\n")
        res.write(str(bc09count12) + "\n")

        for x in cluster12_line:
            for y in BC10_line:
                if x == y:
                    bc10count12 = bc10count12 + 1
        res.write("BC10 Count Cluster 12: \n")
        res.write(str(bc10count12/(len(cluster12_line))) + "\n")
        res.write(str(bc10count12) + "\n")

        for x in cluster12_line:
            for y in BC11_line:
                if x == y:
                    bc11count12 = bc11count12 + 1
        res.write("BC11 Count Cluster 12: \n")
        res.write(str(bc11count12/(len(cluster12_line))) + "\n")
        res.write(str(bc11count12) + "\n")



        ## CLUSTER 13

        cluster13 = open(cluster_13).readlines()
        cluster13_line = []
        for lines in cluster13:
            cluster13_line.append(lines)
        for i in cluster13_line:
            for j in BC01_line:
                if i == j:
                    bc01count13 = bc01count13 + 1
        res.write("BC01 Count Cluster 13: \n")
        res.write(str(bc01count13/(len(cluster13_line))) + " \n")
        res.write(str(bc01count13) + " \n")

        for x in cluster13_line:
            for y in BC02_line:
                if x == y:
                    bc02count13 = bc02count13 + 1
        res.write("BC02 Count Cluster 12: \n")
        res.write(str(bc02count13/(len(cluster13_line))) + "\n")
        res.write(str(bc02count13) + "\n")

        for x in cluster13_line:
            for y in BC03_line:
                if x == y:
                    bc03count13 = bc03count13 + 1
        res.write("BC03 Count Cluster 13: \n")
        res.write(str(bc03count13/(len(cluster13_line))) + "\n")
        res.write(str(bc03count13) + "\n")

        for x in cluster13_line:
            for y in BC03LN_line:
                if x == y:
                    bc03lncount13 = bc03lncount13 + 1
        res.write("BC03 Count Cluster 13: \n")
        res.write(str(bc03lncount13/(len(cluster13_line))) + "\n")
        res.write(str(bc03lncount13) + "\n")

        for x in cluster13_line:
            for y in BC04_line:
                if x == y:
                    bc04count13 = bc04count13 + 1
        res.write("BC04 Count Cluster 13: \n")
        res.write(str(bc04count13/(len(cluster13_line))) + "\n")
        res.write(str(bc04count13) + "\n")

        for x in cluster13_line:
            for y in BC05_line:
                if x == y:
                    bc05count13 = bc05count13 + 1
        res.write("BC05 Count Cluster 13: \n")
        res.write(str(bc05count13/(len(cluster13_line))) + "\n")
        res.write(str(bc05count13) + "\n")

        for x in cluster13_line:
            for y in BC06_line:
                if x == y:
                    bc06count13 = bc06count13 + 1
        res.write("BC06 Count Cluster 13: \n")
        res.write(str(bc06count13/(len(cluster13_line))) + "\n")
        res.write(str(bc06count13) + "\n")

        for x in cluster13_line:
            for y in BC07_line:
                if x == y:
                    bc07count13 = bc07count13 + 1
        res.write("BC07 Count Cluster 13: \n")
        res.write(str(bc07count13/(len(cluster13_line))) + "\n")
        res.write(str(bc07count13) + "\n")

        for x in cluster13_line:
            for y in BC07LN_line:
                if x == y:
                    bc07lncount13 = bc07lncount13 + 1
        res.write("BC07LN Count Cluster 13: \n")
        res.write(str(bc07lncount13/(len(cluster13_line))) + "\n")
        res.write(str(bc07lncount13) + "\n")

        for x in cluster13_line:
            for y in BC08_line:
                if x == y:
                    bc08count13 = bc08count13 + 1
        res.write("BC08 Count Cluster 12: \n")
        res.write(str(bc08count13/(len(cluster13_line))) + "\n")
        res.write(str(bc08count13) + "\n")

        for x in cluster13_line:
            for y in BC09_line:
                if x == y:
                    bc09count13 = bc09count13 + 1
        res.write("BC09 Count Cluster 13: \n")
        res.write(str(bc09count13/(len(cluster13_line))) + "\n")
        res.write(str(bc09count13) + "\n")

        for x in cluster13_line:
            for y in BC10_line:
                if x == y:
                    bc10count13 = bc10count13 + 1
        res.write("BC10 Count Cluster 13: \n")
        res.write(str(bc10count13/(len(cluster13_line))) + "\n")
        res.write(str(bc10count13) + "\n")

        for x in cluster13_line:
            for y in BC11_line:
                if x == y:
                    bc11count13 = bc11count13 + 1
        res.write("BC11 Count Cluster 13: \n")
        res.write(str(bc11count13/(len(cluster13_line))) + "\n")
        res.write(str(bc11count13) + "\n")




    #     dpcount_stats = [dpcount/len(cluster1_line), dpcount2/len(cluster2_line), dpcount3/len(cluster3_line), dpcount4/len(cluster4_line)]
    #     erpcount_stats = [erpcount/len(cluster1_line), erpcount2/len(cluster2_line), erpcount3/len(cluster3_line), erpcount4/len(cluster4_line)]
    #     her2pcount_stats = [her2pcount/len(cluster1_line), her2pcount2/len(cluster2_line), her2pcount3/len(cluster3_line), her2pcount4/len(cluster4_line)]
    #     tnbccount_stats = [tnbccount/len(cluster1_line), tnbccount2/len(cluster2_line), tnbccount3/len(cluster3_line), tnbccount4/len(cluster4_line)]

    #     bc01count_stats = [bc01count1/len(cluster1_line), bc01count2/len(cluster2_line), bc01count3/len(cluster3_line), bc01count4/len(cluster4_line), bc01count5/len(cluster5_line), bc01count6/len(cluster6_line), bc01count7/len(cluster7_line), bc01count8/len(cluster8_line), bc01count9/len(cluster9_line), bc01count10/len(cluster10_line), bc01count11/len(cluster11_line), bc01count12/len(cluster12_line), bc01count13/len(cluster13_line)]
    #     bc02count_stats = [bc02count1/len(cluster1_line), bc02count2/len(cluster2_line), bc02count3/len(cluster3_line), bc02count4/len(cluster4_line), bc02count5/len(cluster5_line), bc02count6/len(cluster6_line), bc02count7/len(cluster7_line), bc02count8/len(cluster8_line), bc02count9/len(cluster9_line), bc02count10/len(cluster10_line), bc02count11/len(cluster11_line), bc02count12/len(cluster12_line), bc02count13/len(cluster13_line)]
    #     bc03count_stats = [bc01count1/len(cluster1_line), bc01count2/len(cluster2_line), bc01count3/len(cluster3_line), bc01count4/len(cluster4_line), bc01count5/len(cluster5_line), bc01count6/len(cluster6_line), bc01count7/len(cluster7_line), bc01count8/len(cluster8_line), bc01count9/len(cluster9_line), bc01count10/len(cluster10_line), bc01count11/len(cluster11_line), bc01count12/len(cluster12_line), bc01count13/len(cluster13_line)]


        bc01count_stats = [bc01count1/len(cluster1_line), bc01count2/len(cluster2_line), bc01count3/len(cluster3_line), bc01count4/len(cluster4_line), bc01count5/len(cluster5_line), bc01count6/len(cluster6_line), bc01count7/len(cluster7_line), bc01count8/len(cluster8_line), bc01count9/len(cluster9_line), bc01count10/len(cluster10_line), bc01count11/len(cluster11_line), bc01count12/len(cluster12_line), bc01count13/len(cluster13_line)]
        bc02count_stats = [bc02count1/len(cluster1_line), bc02count2/len(cluster2_line), bc02count3/len(cluster3_line), bc02count4/len(cluster4_line), bc02count5/len(cluster5_line), bc02count6/len(cluster6_line), bc02count7/len(cluster7_line), bc02count8/len(cluster8_line), bc02count9/len(cluster9_line), bc02count10/len(cluster10_line), bc02count11/len(cluster11_line), bc02count12/len(cluster12_line), bc02count13/len(cluster13_line)]
        bc03count_stats = [bc03count1/len(cluster1_line), bc03count2/len(cluster2_line), bc03count3/len(cluster3_line), bc03count4/len(cluster4_line), bc03count5/len(cluster5_line), bc03count6/len(cluster6_line), bc03count7/len(cluster7_line), bc03count8/len(cluster8_line), bc03count9/len(cluster9_line), bc03count10/len(cluster10_line), bc03count11/len(cluster11_line), bc03count12/len(cluster12_line), bc03count13/len(cluster13_line)]
        bc03lncount_stats = [bc03lncount1/len(cluster1_line), bc03lncount2/len(cluster2_line), bc03lncount3/len(cluster3_line), bc03lncount4/len(cluster4_line), bc03lncount5/len(cluster5_line), bc03lncount6/len(cluster6_line), bc03lncount7/len(cluster7_line), bc03lncount8/len(cluster8_line), bc03lncount9/len(cluster9_line), bc03lncount10/len(cluster10_line), bc03lncount11/len(cluster11_line), bc03lncount12/len(cluster12_line), bc03lncount13/len(cluster13_line)]
        bc04count_stats = [bc04count1/len(cluster1_line), bc04count2/len(cluster2_line), bc04count3/len(cluster3_line), bc04count4/len(cluster4_line), bc04count5/len(cluster5_line), bc04count6/len(cluster6_line), bc04count7/len(cluster7_line), bc04count8/len(cluster8_line), bc04count9/len(cluster9_line), bc04count10/len(cluster10_line), bc04count11/len(cluster11_line), bc04count12/len(cluster12_line), bc04count13/len(cluster13_line)]
        bc05count_stats = [bc05count1/len(cluster1_line), bc05count2/len(cluster2_line), bc05count3/len(cluster3_line), bc05count4/len(cluster4_line), bc05count5/len(cluster5_line), bc05count6/len(cluster6_line), bc05count7/len(cluster7_line), bc05count8/len(cluster8_line), bc05count9/len(cluster9_line), bc05count10/len(cluster10_line), bc05count11/len(cluster11_line), bc05count12/len(cluster12_line), bc05count13/len(cluster13_line)]
        bc06count_stats = [bc06count1/len(cluster1_line), bc06count2/len(cluster2_line), bc06count3/len(cluster3_line), bc06count4/len(cluster4_line), bc06count5/len(cluster5_line), bc06count6/len(cluster6_line), bc06count7/len(cluster7_line), bc06count8/len(cluster8_line), bc06count9/len(cluster9_line), bc06count10/len(cluster10_line), bc06count11/len(cluster11_line), bc06count12/len(cluster12_line), bc06count13/len(cluster13_line)]
        bc07count_stats = [bc07count1/len(cluster1_line), bc07count2/len(cluster2_line), bc07count3/len(cluster3_line), bc07count4/len(cluster4_line), bc07count5/len(cluster5_line), bc07count6/len(cluster6_line), bc07count7/len(cluster7_line), bc07count8/len(cluster8_line), bc07count9/len(cluster9_line), bc07count10/len(cluster10_line), bc07count11/len(cluster11_line), bc07count12/len(cluster12_line), bc07count13/len(cluster13_line)]
        bc07lncount_stats = [bc07lncount1/len(cluster1_line), bc07lncount2/len(cluster2_line), bc07lncount3/len(cluster3_line), bc07lncount4/len(cluster4_line), bc07lncount5/len(cluster5_line), bc07lncount6/len(cluster6_line), bc07lncount7/len(cluster7_line), bc07lncount8/len(cluster8_line), bc07lncount9/len(cluster9_line), bc07lncount10/len(cluster10_line), bc07lncount11/len(cluster11_line), bc07lncount12/len(cluster12_line), bc07lncount13/len(cluster13_line)]
        bc08count_stats = [bc08count1/len(cluster1_line), bc08count2/len(cluster2_line), bc08count3/len(cluster3_line), bc08count4/len(cluster4_line), bc08count5/len(cluster5_line), bc08count6/len(cluster6_line), bc08count7/len(cluster7_line), bc08count8/len(cluster8_line), bc08count9/len(cluster9_line), bc08count10/len(cluster10_line), bc08count11/len(cluster11_line), bc08count12/len(cluster12_line), bc08count13/len(cluster13_line)]
        bc09count_stats = [bc09count1/len(cluster1_line), bc09count2/len(cluster2_line), bc09count3/len(cluster3_line), bc09count4/len(cluster4_line), bc09count5/len(cluster5_line), bc09count6/len(cluster6_line), bc09count7/len(cluster7_line), bc09count8/len(cluster8_line), bc09count9/len(cluster9_line), bc09count10/len(cluster10_line), bc09count11/len(cluster11_line), bc09count12/len(cluster12_line), bc09count13/len(cluster13_line)]
        bc10count_stats = [bc10count1/len(cluster1_line), bc10count2/len(cluster2_line), bc10count3/len(cluster3_line), bc10count4/len(cluster4_line), bc10count5/len(cluster5_line), bc10count6/len(cluster6_line), bc10count7/len(cluster7_line), bc10count8/len(cluster8_line), bc10count9/len(cluster9_line), bc10count10/len(cluster10_line), bc10count11/len(cluster11_line), bc10count12/len(cluster12_line), bc10count13/len(cluster13_line)]
        bc11count_stats = [bc11count1/len(cluster1_line), bc11count2/len(cluster2_line), bc11count3/len(cluster3_line), bc11count4/len(cluster4_line), bc11count5/len(cluster5_line), bc11count6/len(cluster6_line), bc11count7/len(cluster7_line), bc11count8/len(cluster8_line), bc11count9/len(cluster9_line), bc11count10/len(cluster10_line), bc11count11/len(cluster11_line), bc11count12/len(cluster12_line), bc11count13/len(cluster13_line)]




        new_stats = np.zeros((13, 13))

        all_stats = np.array([bc01count_stats, bc02count_stats, bc03count_stats, bc03lncount_stats, bc04count_stats, bc05count_stats, bc06count_stats, bc07count_stats, bc07lncount_stats, bc08count_stats, bc09count_stats, bc10count_stats, bc11count_stats])

        for i in range (0, 13):
            max_num = all_stats.max()
            if(max_num == 0):
                break
            indices = np.where(all_stats == all_stats.max())
            all_stats[indices[0]] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    #         for i in range(0, 13):
    #             all_stats[indices[0], i] = 0
            all_stats[:, indices[1]] = 0
            new_stats[indices] = max_num


        bc01count_stats = new_stats[0]
        bc02count_stats = new_stats[1]
        bc03count_stats = new_stats[2]
        bc03lncount_stats = new_stats[3]
        bc04count_stats = new_stats[4]
        bc05count_stats = new_stats[5]
        bc06count_stats = new_stats[6]
        bc07count_stats = new_stats[7]
        bc07lncount_stats = new_stats[8]
        bc08count_stats = new_stats[9]
        bc09count_stats = new_stats[10]
        bc10count_stats = new_stats[11]
        bc11count_stats = new_stats[12]

        ## CLUSTER 1

        for i in cluster1_line:
            for j in BC01_line:
                    if i == j:
                        if(max(bc01count_stats) == bc01count_stats[0]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC02_line:
                    if i == j:
                        if(max(bc02count_stats) == bc02count_stats[0]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC03_line:
                    if i == j:
                        if(max(bc03count_stats) == bc03count_stats[0]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC03LN_line:
                    if i == j:
                        if(max(bc03lncount_stats) == bc03lncount_stats[0]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC04_line:
                    if i == j:
                        if(max(bc04count_stats) == bc04count_stats[0]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC05_line:
                    if i == j:
                        if(max(bc05count_stats) == bc05count_stats[0]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC06_line:
                    if i == j:
                        if(max(bc06count_stats) == bc06count_stats[0]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC07_line:
                    if i == j:
                        if(max(bc07count_stats) == bc07count_stats[0]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC07LN_line:
                    if i == j:
                        if(max(bc07lncount_stats) == bc07lncount_stats[0]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC08_line:
                    if i == j:
                        if(max(bc08count_stats) == bc08count_stats[0]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC09_line:
                    if i == j:
                        if(max(bc09count_stats) == bc09count_stats[0]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC10_line:
                    if i == j:
                        if(max(bc10count_stats) == bc10count_stats[0]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC11_line:
                    if i == j:
                        if(max(bc11count_stats) == bc11count_stats[0]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})

        ## CLUSTER 2

        for i in cluster2_line:
            for j in BC01_line:
                    if i == j:
                        if(max(bc01count_stats) == bc01count_stats[1]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC02_line:
                    if i == j:
                        if(max(bc02count_stats) == bc02count_stats[1]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC03_line:
                    if i == j:
                        if(max(bc03count_stats) == bc03count_stats[1]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC03LN_line:
                    if i == j:
                        if(max(bc03lncount_stats) == bc03lncount_stats[1]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC04_line:
                    if i == j:
                        if(max(bc04count_stats) == bc04count_stats[1]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC05_line:
                    if i == j:
                        if(max(bc05count_stats) == bc05count_stats[1]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC06_line:
                    if i == j:
                        if(max(bc06count_stats) == bc06count_stats[1]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC07_line:
                    if i == j:
                        if(max(bc07count_stats) == bc07count_stats[1]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC07LN_line:
                    if i == j:
                        if(max(bc07lncount_stats) == bc07lncount_stats[1]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC08_line:
                    if i == j:
                        if(max(bc08count_stats) == bc08count_stats[1]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC09_line:
                    if i == j:
                        if(max(bc09count_stats) == bc09count_stats[1]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC10_line:
                    if i == j:
                        if(max(bc10count_stats) == bc10count_stats[1]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC11_line:
                    if i == j:
                        if(max(bc11count_stats) == bc11count_stats[1]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})

        ## CLUSTER 3

        for i in cluster3_line:
            for j in BC01_line:
                    if i == j:
                        if(max(bc01count_stats) == bc01count_stats[2]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC02_line:
                    if i == j:
                        if(max(bc02count_stats) == bc02count_stats[2]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC03_line:
                    if i == j:
                        if(max(bc03count_stats) == bc03count_stats[2]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC03LN_line:
                    if i == j:
                        if(max(bc03lncount_stats) == bc03lncount_stats[2]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC04_line:
                    if i == j:
                        if(max(bc04count_stats) == bc04count_stats[2]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC05_line:
                    if i == j:
                        if(max(bc05count_stats) == bc05count_stats[2]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC06_line:
                    if i == j:
                        if(max(bc06count_stats) == bc06count_stats[2]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC07_line:
                    if i == j:
                        if(max(bc07count_stats) == bc07count_stats[2]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC07LN_line:
                    if i == j:
                        if(max(bc07lncount_stats) == bc07lncount_stats[2]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC08_line:
                    if i == j:
                        if(max(bc08count_stats) == bc08count_stats[2]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC09_line:
                    if i == j:
                        if(max(bc09count_stats) == bc09count_stats[2]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC10_line:
                    if i == j:
                        if(max(bc10count_stats) == bc10count_stats[2]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC11_line:
                    if i == j:
                        if(max(bc11count_stats) == bc11count_stats[2]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})

        ## CLUSTER 4

        for i in cluster4_line:
            for j in BC01_line:
                    if i == j:
                        if(max(bc01count_stats) == bc01count_stats[3]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC02_line:
                    if i == j:
                        if(max(bc02count_stats) == bc02count_stats[3]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC03_line:
                    if i == j:
                        if(max(bc03count_stats) == bc03count_stats[3]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC03LN_line:
                    if i == j:
                        if(max(bc03lncount_stats) == bc03lncount_stats[3]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC04_line:
                    if i == j:
                        if(max(bc04count_stats) == bc04count_stats[3]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC05_line:
                    if i == j:
                        if(max(bc05count_stats) == bc05count_stats[3]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC06_line:
                    if i == j:
                        if(max(bc06count_stats) == bc06count_stats[3]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC07_line:
                    if i == j:
                        if(max(bc07count_stats) == bc07count_stats[3]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC07LN_line:
                    if i == j:
                        if(max(bc07lncount_stats) == bc07lncount_stats[3]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC08_line:
                    if i == j:
                        if(max(bc08count_stats) == bc08count_stats[3]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC09_line:
                    if i == j:
                        if(max(bc09count_stats) == bc09count_stats[3]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC10_line:
                    if i == j:
                        if(max(bc10count_stats) == bc10count_stats[3]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC11_line:
                    if i == j:
                        if(max(bc11count_stats) == bc11count_stats[3]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})

        ## CLUSTER 5

        for i in cluster5_line:
            for j in BC01_line:
                    if i == j:
                        if(max(bc01count_stats) == bc01count_stats[4]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC02_line:
                    if i == j:
                        if(max(bc02count_stats) == bc02count_stats[4]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC03_line:
                    if i == j:
                        if(max(bc03count_stats) == bc03count_stats[4]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC03LN_line:
                    if i == j:
                        if(max(bc03lncount_stats) == bc03lncount_stats[4]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC04_line:
                    if i == j:
                        if(max(bc04count_stats) == bc04count_stats[4]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC05_line:
                    if i == j:
                        if(max(bc05count_stats) == bc05count_stats[4]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC06_line:
                    if i == j:
                        if(max(bc06count_stats) == bc06count_stats[4]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC07_line:
                    if i == j:
                        if(max(bc07count_stats) == bc07count_stats[4]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC07LN_line:
                    if i == j:
                        if(max(bc07lncount_stats) == bc07lncount_stats[4]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC08_line:
                    if i == j:
                        if(max(bc08count_stats) == bc08count_stats[4]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC09_line:
                    if i == j:
                        if(max(bc09count_stats) == bc09count_stats[4]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC10_line:
                    if i == j:
                        if(max(bc10count_stats) == bc10count_stats[4]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC11_line:
                    if i == j:
                        if(max(bc11count_stats) == bc11count_stats[4]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})

        ## CLUSTER 6

        for i in cluster6_line:
            for j in BC01_line:
                    if i == j:
                        if(max(bc01count_stats) == bc01count_stats[5]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC02_line:
                    if i == j:
                        if(max(bc02count_stats) == bc02count_stats[5]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC03_line:
                    if i == j:
                        if(max(bc03count_stats) == bc03count_stats[5]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC03LN_line:
                    if i == j:
                        if(max(bc03lncount_stats) == bc03lncount_stats[5]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC04_line:
                    if i == j:
                        if(max(bc04count_stats) == bc04count_stats[5]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC05_line:
                    if i == j:
                        if(max(bc05count_stats) == bc05count_stats[5]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC06_line:
                    if i == j:
                        if(max(bc06count_stats) == bc06count_stats[5]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC07_line:
                    if i == j:
                        if(max(bc07count_stats) == bc07count_stats[5]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC07LN_line:
                    if i == j:
                        if(max(bc07lncount_stats) == bc07lncount_stats[5]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC08_line:
                    if i == j:
                        if(max(bc08count_stats) == bc08count_stats[5]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC09_line:
                    if i == j:
                        if(max(bc09count_stats) == bc09count_stats[5]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC10_line:
                    if i == j:
                        if(max(bc10count_stats) == bc10count_stats[5]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC11_line:
                    if i == j:
                        if(max(bc11count_stats) == bc11count_stats[5]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})

        ## CLUSTER 7

        for i in cluster7_line:
            for j in BC01_line:
                    if i == j:
                        if(max(bc01count_stats) == bc01count_stats[6]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC02_line:
                    if i == j:
                        if(max(bc02count_stats) == bc02count_stats[6]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC03_line:
                    if i == j:
                        if(max(bc03count_stats) == bc03count_stats[6]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC03LN_line:
                    if i == j:
                        if(max(bc03lncount_stats) == bc03lncount_stats[6]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC04_line:
                    if i == j:
                        if(max(bc04count_stats) == bc04count_stats[6]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC05_line:
                    if i == j:
                        if(max(bc05count_stats) == bc05count_stats[6]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC06_line:
                    if i == j:
                        if(max(bc06count_stats) == bc06count_stats[6]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC07_line:
                    if i == j:
                        if(max(bc07count_stats) == bc07count_stats[6]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC07LN_line:
                    if i == j:
                        if(max(bc07lncount_stats) == bc07lncount_stats[6]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC08_line:
                    if i == j:
                        if(max(bc08count_stats) == bc08count_stats[6]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC09_line:
                    if i == j:
                        if(max(bc09count_stats) == bc09count_stats[6]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC10_line:
                    if i == j:
                        if(max(bc10count_stats) == bc10count_stats[6]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC11_line:
                    if i == j:
                        if(max(bc11count_stats) == bc11count_stats[6]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})

        ## CLUSTER 8

        for i in cluster8_line:
            for j in BC01_line:
                    if i == j:
                        if(max(bc01count_stats) == bc01count_stats[7]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC02_line:
                    if i == j:
                        if(max(bc02count_stats) == bc02count_stats[7]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC03_line:
                    if i == j:
                        if(max(bc03count_stats) == bc03count_stats[7]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC03LN_line:
                    if i == j:
                        if(max(bc03lncount_stats) == bc03lncount_stats[7]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC04_line:
                    if i == j:
                        if(max(bc04count_stats) == bc04count_stats[7]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC05_line:
                    if i == j:
                        if(max(bc05count_stats) == bc05count_stats[7]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC06_line:
                    if i == j:
                        if(max(bc06count_stats) == bc06count_stats[7]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC07_line:
                    if i == j:
                        if(max(bc07count_stats) == bc07count_stats[7]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC07LN_line:
                    if i == j:
                        if(max(bc07lncount_stats) == bc07lncount_stats[7]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC08_line:
                    if i == j:
                        if(max(bc08count_stats) == bc08count_stats[7]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC09_line:
                    if i == j:
                        if(max(bc09count_stats) == bc09count_stats[7]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC10_line:
                    if i == j:
                        if(max(bc10count_stats) == bc10count_stats[7]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC11_line:
                    if i == j:
                        if(max(bc11count_stats) == bc11count_stats[7]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})

        ## CLUSTER 9

        for i in cluster9_line:
            for j in BC01_line:
                    if i == j:
                        if(max(bc01count_stats) == bc01count_stats[8]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC02_line:
                    if i == j:
                        if(max(bc02count_stats) == bc02count_stats[8]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC03_line:
                    if i == j:
                        if(max(bc03count_stats) == bc03count_stats[8]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC03LN_line:
                    if i == j:
                        if(max(bc03lncount_stats) == bc03lncount_stats[8]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC04_line:
                    if i == j:
                        if(max(bc04count_stats) == bc04count_stats[8]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC05_line:
                    if i == j:
                        if(max(bc05count_stats) == bc05count_stats[8]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC06_line:
                    if i == j:
                        if(max(bc06count_stats) == bc06count_stats[8]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC07_line:
                    if i == j:
                        if(max(bc07count_stats) == bc07count_stats[8]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC07LN_line:
                    if i == j:
                        if(max(bc07lncount_stats) == bc07lncount_stats[8]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC08_line:
                    if i == j:
                        if(max(bc08count_stats) == bc08count_stats[8]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC09_line:
                    if i == j:
                        if(max(bc09count_stats) == bc09count_stats[8]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC10_line:
                    if i == j:
                        if(max(bc10count_stats) == bc10count_stats[8]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC11_line:
                    if i == j:
                        if(max(bc11count_stats) == bc11count_stats[8]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})

        ## CLUSTER 10

        for i in cluster10_line:
            for j in BC01_line:
                    if i == j:
                        if(max(bc01count_stats) == bc01count_stats[9]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC02_line:
                    if i == j:
                        if(max(bc02count_stats) == bc02count_stats[9]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC03_line:
                    if i == j:
                        if(max(bc03count_stats) == bc03count_stats[9]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC03LN_line:
                    if i == j:
                        if(max(bc03lncount_stats) == bc03lncount_stats[9]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC04_line:
                    if i == j:
                        if(max(bc04count_stats) == bc04count_stats[9]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC05_line:
                    if i == j:
                        if(max(bc05count_stats) == bc05count_stats[9]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC06_line:
                    if i == j:
                        if(max(bc06count_stats) == bc06count_stats[9]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC07_line:
                    if i == j:
                        if(max(bc07count_stats) == bc07count_stats[9]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC07LN_line:
                    if i == j:
                        if(max(bc07lncount_stats) == bc07lncount_stats[9]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC08_line:
                    if i == j:
                        if(max(bc08count_stats) == bc08count_stats[9]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC09_line:
                    if i == j:
                        if(max(bc09count_stats) == bc09count_stats[9]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC10_line:
                    if i == j:
                        if(max(bc10count_stats) == bc10count_stats[9]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC11_line:
                    if i == j:
                        if(max(bc11count_stats) == bc11count_stats[9]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})

        ## CLUSTER 11

        for i in cluster11_line:
            for j in BC01_line:
                    if i == j:
                        if(max(bc01count_stats) == bc01count_stats[10]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC02_line:
                    if i == j:
                        if(max(bc02count_stats) == bc02count_stats[10]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC03_line:
                    if i == j:
                        if(max(bc03count_stats) == bc03count_stats[10]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC03LN_line:
                    if i == j:
                        if(max(bc03lncount_stats) == bc03lncount_stats[10]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC04_line:
                    if i == j:
                        if(max(bc04count_stats) == bc04count_stats[10]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC05_line:
                    if i == j:
                        if(max(bc05count_stats) == bc05count_stats[10]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC06_line:
                    if i == j:
                        if(max(bc06count_stats) == bc06count_stats[10]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC07_line:
                    if i == j:
                        if(max(bc07count_stats) == bc07count_stats[10]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC07LN_line:
                    if i == j:
                        if(max(bc07lncount_stats) == bc07lncount_stats[10]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC08_line:
                    if i == j:
                        if(max(bc08count_stats) == bc08count_stats[10]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC09_line:
                    if i == j:
                        if(max(bc09count_stats) == bc09count_stats[10]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC10_line:
                    if i == j:
                        if(max(bc10count_stats) == bc10count_stats[10]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC11_line:
                    if i == j:
                        if(max(bc11count_stats) == bc11count_stats[10]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})

        ## CLUSTER 12


        for i in cluster12_line:
            for j in BC01_line:
                    if i == j:
                        if(max(bc01count_stats) == bc01count_stats[11]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC02_line:
                    if i == j:
                        if(max(bc02count_stats) == bc02count_stats[11]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC03_line:
                    if i == j:
                        if(max(bc03count_stats) == bc03count_stats[11]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC03LN_line:
                    if i == j:
                        if(max(bc03lncount_stats) == bc03lncount_stats[11]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC04_line:
                    if i == j:
                        if(max(bc04count_stats) == bc04count_stats[11]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC05_line:
                    if i == j:
                        if(max(bc05count_stats) == bc05count_stats[11]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC06_line:
                    if i == j:
                        if(max(bc06count_stats) == bc06count_stats[11]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC07_line:
                    if i == j:
                        if(max(bc07count_stats) == bc07count_stats[11]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC07LN_line:
                    if i == j:
                        if(max(bc07lncount_stats) == bc07lncount_stats[11]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC08_line:
                    if i == j:
                        if(max(bc08count_stats) == bc08count_stats[11]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC09_line:
                    if i == j:
                        if(max(bc09count_stats) == bc09count_stats[11]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC10_line:
                    if i == j:
                        if(max(bc10count_stats) == bc10count_stats[11]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC11_line:
                    if i == j:
                        if(max(bc11count_stats) == bc11count_stats[11]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})

        ## CLUSTER 13

        for i in cluster13_line:
            for j in BC01_line:
                    if i == j:
                        if(max(bc01count_stats) == bc01count_stats[12]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC02_line:
                    if i == j:
                        if(max(bc02count_stats) == bc02count_stats[12]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC03_line:
                    if i == j:
                        if(max(bc03count_stats) == bc03count_stats[12]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC03LN_line:
                    if i == j:
                        if(max(bc03lncount_stats) == bc03lncount_stats[12]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC04_line:
                    if i == j:
                        if(max(bc04count_stats) == bc04count_stats[12]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC05_line:
                    if i == j:
                        if(max(bc05count_stats) == bc05count_stats[12]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC06_line:
                    if i == j:
                        if(max(bc06count_stats) == bc06count_stats[12]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC07_line:
                    if i == j:
                        if(max(bc07count_stats) == bc07count_stats[12]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC07LN_line:
                    if i == j:
                        if(max(bc07lncount_stats) == bc07lncount_stats[12]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC08_line:
                    if i == j:
                        if(max(bc08count_stats) == bc08count_stats[12]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC09_line:
                    if i == j:
                        if(max(bc09count_stats) == bc09count_stats[12]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC10_line:
                    if i == j:
                        if(max(bc10count_stats) == bc10count_stats[12]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})
            for j in BC11_line:
                    if i == j:
                        if(max(bc11count_stats) == bc11count_stats[12]): 
                            color = 'black'
                        else: 
                            color = 'red'
                        truthnum1.update({true_coords[i[0:len(i)-1]]:color})

        correctCount = 0
        for i in truthnum1:
            if(truthnum1[i] == 'black'):
                correctCount += 1
        incorrectCount = (len(truthnum1)-correctCount)/len(truthnum1)
        correctCount = correctCount/len(truthnum1)
        return truthnum1, correctCount*100, incorrectCount*100
    def DataViz6(figname, algorithms, cluster1, cluster2, cluster3, cluster4, cluster5, cluster6, cluster7, cluster8, cluster9, cluster10, cluster11, cluster12, cluster13, results):
        for i in range(len(algorithms)):
            truthnum1, correct, incorrect = DataCollectionPatientTvP(algorithms[i], cluster1[i], cluster2[i], cluster3[i], cluster4[i], cluster5[i], cluster6[i], cluster7[i], cluster8[i], cluster9[i], cluster10[i], cluster11[i], cluster12[i], cluster13[i], results[i])
            if(algorithms[i] == gmm):
                global gmmAverage
                gmmAverage += correct
                document.write('GAUSSIAN: ' + str(correct) +'\n')
                print('GAUSSIAN TRIAL ' + str(b) + ': ' + str(correct) +'\n')
            elif(algorithms[i] == ward):
                global wardAverage
                wardAverage += correct
                document.write('WARD: ' + str(correct) +'\n')
                print('WARD TRIAL ' + str(b) + ': ' + str(correct) +'\n')
            elif(algorithms[i] == birch):
                global birchAverage
                birchAverage += correct
                document.write('BIRCH: ' + str(correct) +'\n')
                print('BIRCH TRIAL ' + str(b) + ': ' + str(correct) +'\n')
            elif(algorithms[i] == affinity_propagation):
                global affPropAverage
                affPropAverage += correct
                document.write('AFFINITY PROPAGATION: ' + str(correct) +'\n')
                print('AFFINITY PROPAGATION TRIAL ' + str(b) + ': ' + str(correct) +'\n')
            elif(algorithms[i] == average_linkage):
                global aggAverage
                aggAverage += correct
                document.write('AGGLOMERATIVE: ' + str(correct) +'\n')
                print('AGGLOMERATIVE TRIAL ' + str(b) + ': ' + str(correct) +'\n')
            elif(algorithms[i] == spectral):
                global spectralAverage
                spectralAverage += correct
                document.write('SPECTRAL: ' + str(correct) +'\n')
                print('SPECTRAL TRIAL ' + str(b) + ': ' + str(correct) +'\n')
            elif(algorithms[i] == kmeans):
                global kmAverage
                kmAverage += correct
                document.write('KMEANS: ' + str(correct) + '\n')
                print('KMEANS TRIAL ' + str(b) + ': ' + str(correct) + '\n')
           
    algorithms = [gmm, spectral, ward, birch, average_linkage, kmeans]
    figname = ["Gaussian Mixture Model", "Spectral Clustering", "Ward Clustering", "Birch Clustering", "Agglomerative Clustering", "KMeans Clustering"]
    cluster1 = ["gmm_cluster1.txt", "spectral_cluster1.txt", "ward_cluster1.txt", "birch_cluster1.txt", "agg_cluster1.txt", "km_cluster1.txt"]
    cluster2 = ["gmm_cluster2.txt", "spectral_cluster2.txt", "ward_cluster2.txt", "birch_cluster2.txt", "agg_cluster2.txt", "km_cluster2.txt"]
    cluster3 = ["gmm_cluster3.txt", "spectral_cluster3.txt", "ward_cluster3.txt", "birch_cluster3.txt", "agg_cluster3.txt", "km_cluster3.txt"]
    cluster4 = ["gmm_cluster4.txt", "spectral_cluster4.txt", "ward_cluster4.txt", "birch_cluster4.txt", "agg_cluster4.txt", "km_cluster4.txt"]
    cluster5 = ["gmm_cluster5.txt", "spectral_cluster5.txt", "ward_cluster5.txt", "birch_cluster5.txt", "agg_cluster5.txt", "km_cluster5.txt"]
    cluster6 = ["gmm_cluster6.txt", "spectral_cluster6.txt", "ward_cluster6.txt", "birch_cluster6.txt", "agg_cluster6.txt", "km_cluster6.txt"]
    cluster7 = ["gmm_cluster7.txt", "spectral_cluster7.txt", "ward_cluster7.txt", "birch_cluster7.txt", "agg_cluster7.txt", "km_cluster7.txt"]
    cluster8 = ["gmm_cluster8.txt", "spectral_cluster8.txt", "ward_cluster8.txt", "birch_cluster8.txt", "agg_cluster8.txt", "km_cluster8.txt"]
    cluster9 = ["gmm_cluster9.txt", "spectral_cluster9.txt", "ward_cluster9.txt", "birch_cluster9.txt", "agg_cluster9.txt", "km_cluster9.txt"]
    cluster10 = ["gmm_cluster10.txt", "spectral_cluster10.txt", "ward_cluster10.txt", "birch_cluster10.txt", "agg_cluster10.txt", "km_cluster10.txt"]
    cluster11 = ["gmm_cluster11.txt", "spectral_cluster11.txt", "ward_cluster11.txt", "birch_cluster11.txt", "agg_cluster11.txt", "km_cluster11.txt"]
    cluster12 = ["gmm_cluster12.txt", "spectral_cluster12.txt", "ward_cluster12.txt", "birch_cluster12.txt", "agg_cluster12.txt", "km_cluster12.txt"]
    cluster13 = ["gmm_cluster13.txt", "spectral_cluster13.txt", "ward_cluster13.txt", "birch_cluster13.txt", "agg_cluster13.txt", "km_cluster13.txt"]
    results = ["gmm_results.txt", "spectral_results.txt", "ward_results.txt", "birch_results.txt", "agg_results.txt", "km_results.txt"]
    
    DataViz6(figname, algorithms, cluster1, cluster2, cluster3, cluster4, cluster5, cluster6, cluster7, cluster8, cluster9, cluster10, cluster11, cluster12, cluster13, results)
document.write('GAUSSIAN FINAL: ' + str(gmmAverage/100) +'\n')
document.write('BIRCH FINAL: ' + str(birchAverage/100) +'\n')
document.write('WARD FINAL: ' + str(wardAverage/100) +'\n')
document.write('SPECTRAL FINAL: ' + str(spectralAverage/100) +'\n')
document.write('AGGLOMERATIVE FINAL: ' + str(aggAverage/100) +'\n')
document.write('KMEANS FINAL: ' + str(kmAverage/100) + '\n')
document.close()
print('GAUSSIAN FINAL: ' + str(gmmAverage/100) +'\n')
print('BIRCH FINAL: ' + str(birchAverage/100) +'\n')
print('WARD FINAL: ' + str(wardAverage/100) +'\n')
print('SPECTRAL FINAL: ' + str(spectralAverage/100) +'\n')
print('AGGLOMERATIVE FINAL: ' + str(aggAverage/100) +'\n')
print('KMEANS FINAL: ' + str(kmAverage/100) + '\n')

GAUSSIAN TRIAL 1: 55.03875968992248

SPECTRAL TRIAL 1: 56.58914728682171

WARD TRIAL 1: 58.52713178294574

BIRCH TRIAL 1: 55.62015503875969

AGGLOMERATIVE TRIAL 1: 45.542635658914726

KMEANS TRIAL 1: 56.395348837209305

GAUSSIAN TRIAL 2: 54.263565891472865

SPECTRAL TRIAL 2: 43.02325581395349

WARD TRIAL 2: 56.97674418604651

BIRCH TRIAL 2: 55.23255813953488

AGGLOMERATIVE TRIAL 2: 50.1937984496124

KMEANS TRIAL 2: 56.78294573643411

GAUSSIAN TRIAL 3: 48.837209302325576

SPECTRAL TRIAL 3: 46.31782945736434

WARD TRIAL 3: 59.68992248062015

BIRCH TRIAL 3: 56.007751937984494

AGGLOMERATIVE TRIAL 3: 40.89147286821706

KMEANS TRIAL 3: 56.78294573643411

GAUSSIAN TRIAL 4: 55.03875968992248

SPECTRAL TRIAL 4: 48.643410852713174

WARD TRIAL 4: 54.263565891472865

BIRCH TRIAL 4: 55.42635658914728

AGGLOMERATIVE TRIAL 4: 40.310077519379846

KMEANS TRIAL 4: 55.42635658914728

GAUSSIAN TRIAL 5: 51.356589147286826

SPECTRAL TRIAL 5: 60.85271317829457

WARD TRIAL 5: 57.36434108527132

BIRCH TRIAL 5

AGGLOMERATIVE TRIAL 37: 34.10852713178294

KMEANS TRIAL 37: 56.78294573643411

GAUSSIAN TRIAL 38: 57.36434108527132

SPECTRAL TRIAL 38: 56.58914728682171

WARD TRIAL 38: 56.395348837209305

BIRCH TRIAL 38: 57.55813953488372

AGGLOMERATIVE TRIAL 38: 46.12403100775194

KMEANS TRIAL 38: 57.751937984496124

GAUSSIAN TRIAL 39: 59.10852713178295

SPECTRAL TRIAL 39: 41.86046511627907

WARD TRIAL 39: 59.30232558139535

BIRCH TRIAL 39: 58.333333333333336

AGGLOMERATIVE TRIAL 39: 49.8062015503876

KMEANS TRIAL 39: 55.81395348837209

GAUSSIAN TRIAL 40: 50.58139534883721

SPECTRAL TRIAL 40: 52.32558139534884

WARD TRIAL 40: 42.63565891472868

BIRCH TRIAL 40: 57.55813953488372

AGGLOMERATIVE TRIAL 40: 53.100775193798455

KMEANS TRIAL 40: 56.395348837209305

GAUSSIAN TRIAL 41: 57.94573643410853

SPECTRAL TRIAL 41: 44.37984496124031

WARD TRIAL 41: 58.720930232558146

BIRCH TRIAL 41: 44.96124031007752

AGGLOMERATIVE TRIAL 41: 42.63565891472868

KMEANS TRIAL 41: 58.52713178294574

GAUSSIAN TRIAL 42: 5

GAUSSIAN TRIAL 74: 54.65116279069767

SPECTRAL TRIAL 74: 55.42635658914728

WARD TRIAL 74: 57.94573643410853

BIRCH TRIAL 74: 58.52713178294574

AGGLOMERATIVE TRIAL 74: 39.922480620155035

KMEANS TRIAL 74: 59.10852713178295

GAUSSIAN TRIAL 75: 56.58914728682171

SPECTRAL TRIAL 75: 38.372093023255815

WARD TRIAL 75: 42.248062015503876

BIRCH TRIAL 75: 56.395348837209305

AGGLOMERATIVE TRIAL 75: 45.93023255813954

KMEANS TRIAL 75: 55.62015503875969

GAUSSIAN TRIAL 76: 49.8062015503876

SPECTRAL TRIAL 76: 60.65891472868217

WARD TRIAL 76: 56.97674418604651

BIRCH TRIAL 76: 57.36434108527132

AGGLOMERATIVE TRIAL 76: 49.8062015503876

KMEANS TRIAL 76: 57.94573643410853

GAUSSIAN TRIAL 77: 59.10852713178295

SPECTRAL TRIAL 77: 48.06201550387597

WARD TRIAL 77: 59.496124031007746

BIRCH TRIAL 77: 43.21705426356589

AGGLOMERATIVE TRIAL 77: 47.286821705426355

KMEANS TRIAL 77: 59.496124031007746

GAUSSIAN TRIAL 78: 59.496124031007746

SPECTRAL TRIAL 78: 46.12403100775194

WARD TRIAL 78: 58.1395